<a href="https://colab.research.google.com/github/bulentyasan-sys/tradingview-ai-signals/blob/main/6kasimtupras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install yahooquery --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00


In [23]:
from yahooquery import Ticker
import pandas as pd

tickers = [
    "AKBNK.IS","ARCLK.IS","ASELS.IS","ASTOR.IS","BIMAS.IS","FROTO.IS","GARAN.IS",
    "HEKTS.IS","ISCTR.IS","KCHOL.IS","KOZAA.IS","KOZAL.IS","KRDMD.IS","PGSUS.IS",
    "PETKM.IS","SAHOL.IS","SASA.IS","SISE.IS","TCELL.IS","THYAO.IS","TOASO.IS",
    "TUKAS.IS","TTKOM.IS","TUPRS.IS","VESTL.IS","YKBNK.IS","ALARK.IS","ENJSA.IS",
    "EUREN.IS","SNGYO.IS"
]

all_rows = []

for t in tickers:
    print("Downloading:", t)
    data = Ticker(t).history(start="2022-01-01", end="2025-01-01")
    data = data.reset_index()
    data["Ticker"] = t.replace(".IS","")
    all_rows.append(data)

df = pd.concat(all_rows, ignore_index=True)

df = df[['date','Ticker','open','high','low','close','volume']]
df.columns = ['Date','Ticker','Open','High','Low','Close','Volume']

df.to_csv('/content/BIST30_LONG.csv', index=False)

print("✅ DONE → /content/BIST30_LONG.csv")
print(df.shape)
print(df.head())


Downloading: AKBNK.IS
Downloading: ARCLK.IS
Downloading: ASELS.IS
Downloading: ASTOR.IS
Downloading: BIMAS.IS
Downloading: FROTO.IS
Downloading: GARAN.IS
Downloading: HEKTS.IS
Downloading: ISCTR.IS
Downloading: KCHOL.IS
Downloading: KOZAA.IS
Downloading: KOZAL.IS
Downloading: KRDMD.IS
Downloading: PGSUS.IS
Downloading: PETKM.IS
Downloading: SAHOL.IS
Downloading: SASA.IS
Downloading: SISE.IS
Downloading: TCELL.IS
Downloading: THYAO.IS
Downloading: TOASO.IS
Downloading: TUKAS.IS
Downloading: TTKOM.IS
Downloading: TUPRS.IS
Downloading: VESTL.IS
Downloading: YKBNK.IS
Downloading: ALARK.IS
Downloading: ENJSA.IS
Downloading: EUREN.IS
Downloading: SNGYO.IS
✅ DONE → /content/BIST30_LONG.csv
(22134, 7)
         Date Ticker  Open  High   Low  Close       Volume
0  2022-01-03  AKBNK  7.20  7.37  7.18   7.33   94514821.0
1  2022-01-04  AKBNK  7.33  7.44  7.17   7.27  121714202.0
2  2022-01-05  AKBNK  7.27  7.35  7.21   7.34   83950950.0
3  2022-01-06  AKBNK  7.28  7.40  7.25   7.32  115505011.0
4 

In [26]:
import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# --- Load clean long-format BIST30 ---
df = pd.read_csv('/content/BIST30_LONG.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Ticker','Date']).reset_index(drop=True)

# --- TECHNICAL INDICATORS ---
def add_ind(g):
    g = g.copy()

    # EMA
    g['EMA9']  = g['Close'].ewm(span=9).mean()
    g['EMA21'] = g['Close'].ewm(span=21).mean()
    g['EMA50'] = g['Close'].ewm(span=50).mean()

    # RSI14
    delta = g['Close'].diff()
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    rs = gain.rolling(14).mean() / (loss.rolling(14).mean() + 1e-9)
    g['RSI14'] = 100 - (100/(1+rs))

    # MACD Histogram
    ema12 = g['Close'].ewm(span=12).mean()
    ema26 = g['Close'].ewm(span=26).mean()
    macd = ema12 - ema26
    g['MACDHist'] = macd - macd.ewm(span=9).mean()

    # Volume SMAs
    g['Vol_SMA2']  = g['Volume'].rolling(2).mean()
    g['Vol_SMA10'] = g['Volume'].rolling(10).mean()

    # Normalized ATR
    prev = g['Clo]()


SyntaxError: unterminated string literal (detected at line 38) (ipython-input-4104071857.py, line 38)

In [27]:
import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

df = pd.read_csv('/content/BIST30_LONG.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Ticker','Date']).reset_index(drop=True)


In [30]:
def add_ind(g):
    g = g.copy()
    g['EMA9']  = g['Close'].ewm(span=9).mean()
    g['EMA21'] = g['Close'].ewm(span=21).mean()
    g['EMA50'] = g['Close'].ewm(span=50).mean()

    delta = g['Close'].diff()
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    rs = gain.rolling(14).mean() / (loss.rolling(14).mean() + 1e-9)
    g['RSI14'] = 100 - (100/(1+rs))

    ema12 = g['Close'].ewm(span=12).mean()
    ema26 = g['Close'].ewm(span=26).mean()
    macd = ema12 - ema26
    g['MACDHist'] = macd - macd.ewm(span=9).mean()

    g['Vol_SMA2']  = g['Volume'].rolling(2).mean()
    g['Vol_SMA10'] = g['Volume'].rolling(10).mean()

    prev_close = g['Close'].shift()
    tr = pd.concat([
        (g['High'] - g['Low']).abs(),
        (g['High'] - prev_close).abs(),
        (g['Low']  - prev_close).abs()
    ], axis=1).max(axis=1)

    g['nATR'] = tr.rolling(14).mean() / g['Close']
    return g


In [31]:
parts = []
for t, g in df.groupby('Ticker'):
    enriched = add_ind(g)
    enriched = enriched.iloc[60:]
    parts.append(enriched)

dfi = pd.concat(parts, ignore_index=True)


In [32]:
dfi['Target_2d'] = dfi.groupby('Ticker')['Close'].shift(-2) / dfi['Close'] - 1
dfi['Label'] = (dfi['Target_2d'] >= 0.015).astype(int)

dfi = dfi.dropna(subset=['Label'])


In [33]:
features = ['EMA9','EMA21','EMA50','RSI14','MACDHist','Vol_SMA2','Vol_SMA10','nATR']

train = dfi[dfi['Date'] < '2024-01-01']
valid = dfi[(dfi['Date'] >= '2024-01-01') & (dfi['Date'] <= '2024-12-31')]

X_train = train[features].values
y_train = train['Label'].values

X_valid = valid[features].values
y_valid = valid['Label'].values


In [34]:
clf = HistGradientBoostingClassifier(max_depth=6, learning_rate=0.05)
clf.fit(X_train, y_train)

proba = clf.predict_proba(X_valid)[:,1]
pred  = (proba >= 0.50).astype(int)


In [35]:
print("TRAIN:", train.shape)
print("VALID:", valid.shape)

print("\nVALID LABEL COUNTS:")
print(valid['Label'].value_counts())

print("\nMETRICS:")
print("Accuracy :", accuracy_score(y_valid,pred))
print("Precision:", precision_score(y_valid,pred,zero_division=0))
print("Recall   :", recall_score(y_valid,pred,zero_division=0))
print("AUC      :", roc_auc_score(y_valid,proba))


TRAIN: (12858, 17)
VALID: (7476, 17)

VALID LABEL COUNTS:
Label
0    4980
1    2496
Name: count, dtype: int64

METRICS:
Accuracy : 0.6599785981808454
Precision: 0.3357142857142857
Recall   : 0.018830128205128204
AUC      : 0.5210666383482649


In [36]:
valid2 = valid.copy()
valid2['ML_Proba'] = proba

def tech_ok(r):
    return (
        r['EMA9'] > r['EMA21'] > r['EMA50'] and
        45 <= r['RSI14'] <= 65 and
        r['MACDHist'] > 0 and
        r['Vol_SMA2'] > r['Vol_SMA10'] and
        0.005 < r['nATR'] < 0.08
    )

valid2['Tech_OK'] = valid2.apply(tech_ok, axis=1)
valid2['HybridScore'] = valid2['ML_Proba'] + valid2['Tech_OK'].astype(int)*0.2

last_day = valid2['Date'].max()
signals = valid2[valid2['Date'] == last_day].sort_values('HybridScore', ascending=False)

print("\nSignals for last day:")
print(signals[['Date','Ticker','Close','ML_Proba','Tech_OK','HybridScore']].head(20))



Signals for last day:
            Date Ticker       Close  ML_Proba  Tech_OK  HybridScore
9289  2024-12-31  KOZAA   74.250000  0.344294     True     0.544294
3875  2024-12-31  BIMAS  528.500000  0.298015     True     0.498015
7909  2024-12-31  ISCTR   13.540000  0.458688    False     0.458688
16192 2024-12-31  THYAO  281.000000  0.451337    False     0.451337
14812 2024-12-31  SNGYO    3.140000  0.447983    False     0.447983
18953 2024-12-31  TUPRS  141.899994  0.445244    False     0.445244
7219  2024-12-31  HEKTS    3.870000  0.439892    False     0.439892
13431 2024-12-31   SASA    4.070000  0.439891    False     0.439891
20333 2024-12-31  YKBNK   30.620001  0.429704    False     0.429704
8599  2024-12-31  KCHOL  178.600006  0.404636    False     0.404636
9979  2024-12-31  KOZAL   22.580000  0.373679    False     0.373679
10669 2024-12-31  KRDMD   27.100000  0.369897    False     0.369897
4565  2024-12-31  ENJSA   58.900002  0.361399    False     0.361399
3185  2024-12-31  ASTOR  

In [37]:
import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# === LOAD CLEAN BIST30 DATA ===
df = pd.read_csv('/content/BIST30_LONG.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Ticker','Date']).reset_index(drop=True)


# === FEATURE ENGINEERING V7 (MEDIUM) ===
def add_features(g):

    g = g.copy()

    # Momentum Returns
    g['Ret1'] = g['Close'].pct_change(1)
    g['Ret2'] = g['Close'].pct_change(2)
    g['Ret5'] = g['Close'].pct_change(5)

    # Price levels
    g['HL2']  = (g['High'] + g['Low']) / 2
    g['HLC3'] = (g['High'] + g['Low'] + g['Close']) / 3

    # EMA trends
    g['EMA9']  = g['Close'].ewm(span=9).mean()
    g['EMA21'] = g['Close'].ewm(span=21).mean()
    g['EMA50'] = g['Close'].ewm(span=50).mean()

    # EMA slope
    g['EMA_Slope'] = g['EMA21'] - g['EMA50']

    # RSI14
    delta = g['Close'].diff()
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    rs = gain.rolling(14).mean() / (loss.rolling(14).mean() + 1e-9)
    g['RSI14'] = 100 - (100/(1+rs))

    # MACD Histogram components
    ema12 = g['Close'].ewm(span=12).mean()
    ema26 = g['Close'].ewm(span=26).mean()
    macd = ema12 - ema26
    g['MACDHist'] = macd - macd.ewm(span=9).mean()

    # Volume anomalies
    g['Vol_SMA2']  = g['Volume'].rolling(2).mean()
    g['Vol_SMA10'] = g['Volume'].rolling(10).mean()
    g['Vol_Z']     = (g['Volume'] - g['Vol_SMA10']) / (g['Vol_SMA10'] + 1e-9)

    # Normalized ATR
    prev_close = g['Close'].shift()
    tr = pd.concat([
        (g['High'] - g['Low']).abs(),
        (g['High'] - prev_close).abs(),
        (g['Low']  - prev_close).abs()
    ], axis=1).max(axis=1)
    g['nATR'] = tr.rolling(14).mean() / g['Close']

    return g


# === APPLY FEATURES TO ALL TICKERS ===
parts = []
for t, g in df.groupby('Ticker'):
    enriched = add_features(g)
    enriched = enriched.iloc[60:]  # remove warm-up
    parts.append(enriched)

dfi = pd.concat(parts, ignore_index=True)

# === LABEL: 2-Day Forward Return >= +1.5% ===
dfi['Target_2d'] = dfi.groupby('Ticker')['Close'].shift(-2) / dfi['Close'] - 1
dfi['Label'] = (dfi['Target_2d'] >= 0.015).astype(int)
dfi = dfi.dropna(subset=['Label'])

# === FEATURE LIST (MEDIUM FE v7) ===
features = [
    'Ret1','Ret2','Ret5',
    'HL2','HLC3',
    'EMA9','EMA21','EMA50','EMA_Slope',
    'RSI14','MACDHist',
    'Vol_SMA2','Vol_SMA10','Vol_Z',
    'nATR'
]

# === TRAIN / VALID SPLIT ===
train = dfi[dfi['Date'] < '2024-01-01']
valid = dfi[(dfi['Date'] >= '2024-01-01') & (dfi['Date'] <= '2024-12-31')]

X_train = train[features].fillna(0).values
y_train = train['Label'].values

X_valid = valid[features].fillna(0).values
y_valid = valid['Label'].values

# === MODEL — HIGH WR SETTINGS ===
clf = HistGradientBoostingClassifier(
    max_depth=6,
    learning_rate=0.04,
    max_iter=300,
    l2_regularization=0.5
)
clf.fit(X_train, y_train)

proba = clf.predict_proba(X_valid)[:,1]

# THRESHOLD FOR HIGH WR MODE
pred = (proba >= 0.70).astype(int)

# === METRICS ===
print("TRAIN:", train.shape)
print("VALID:", valid.shape)

print("\nVALID LABEL COUNTS:")
print(valid['Label'].value_counts())

print("\nMETRICS:")
print("Accuracy :", accuracy_score(y_valid,pred))
print("Precision:", precision_score(y_valid,pred,zero_division=0))
print("Recall   :", recall_score(y_valid,pred,zero_division=0))
print("AUC      :", roc_auc_score(y_valid,proba))


# === HYBRID SCORING (HIGH WR VERSION) ===
valid2 = valid.copy()
valid2['ML_Proba'] = proba

def tech_ok(r):
    return (
        r['Close'] > r['EMA21'] >
        r['EMA50'] and
        r['RSI14'] >= 52 and r['RSI14'] <= 62 and
        r['MACDHist'] > 0 and
        r['Vol_SMA2'] > 1.1 * r['Vol_SMA10'] and
        r['nATR'] >= 0.005 and r['nATR'] <= 0.06
    )

valid2['Tech_OK'] = valid2.apply(tech_ok, axis=1)

valid2['HybridScore'] = (
    valid2['ML_Proba'] * 0.70 +
    valid2['Tech_OK'].astype(int) * 0.30
)

# === FINAL SIGNALS ===
last_day = valid2['Date'].max()
signals = valid2[valid2['Date'] == last_day].sort_values('HybridScore', ascending=False)

cols = ['Date','Ticker','Close','ML_Proba','Tech_OK','HybridScore']
print("\n=== HIGH WR SIGNALS FOR LAST DAY ===")
print(signals[cols].head(20))

signals[cols].to_csv('/content/BIST30_HIGHR_SIGNALS.csv', index=False)
print("\n✅ Saved: /content/BIST30_HIGHR_SIGNALS.csv")


TRAIN: (12858, 24)
VALID: (7476, 24)

VALID LABEL COUNTS:
Label
0    4980
1    2496
Name: count, dtype: int64

METRICS:
Accuracy : 0.6654628143392188
Precision: 0.2727272727272727
Recall   : 0.001201923076923077
AUC      : 0.5059180632787561

=== HIGH WR SIGNALS FOR LAST DAY ===
            Date Ticker       Close  ML_Proba  Tech_OK  HybridScore
9289  2024-12-31  KOZAA   74.250000  0.419896     True     0.593927
7909  2024-12-31  ISCTR   13.540000  0.561041    False     0.392729
13431 2024-12-31   SASA    4.070000  0.520232    False     0.364162
14812 2024-12-31  SNGYO    3.140000  0.514427    False     0.360099
5148  2024-12-31  EUREN    4.890000  0.424341    False     0.297039
20333 2024-12-31  YKBNK   30.620001  0.415467    False     0.290827
7219  2024-12-31  HEKTS    3.870000  0.409582    False     0.286708
6529  2024-12-31  GARAN  124.400002  0.404344    False     0.283041
18263 2024-12-31  TUKAS    7.020000  0.401853    False     0.281297
11359 2024-12-31  PETKM   18.100000  0.3

In [38]:
# ============================
# HIGH-WR BACKTEST v1.0 (1B / 2C / 3B)
# - Entry: next day OPEN
# - Exit: TP +1.5% or SL -1%, else max 2 bars then exit at day2 CLOSE
# - Per day: take ONLY the top HybridScore signal (3B)
# ============================

import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# ---------- Load & prep ----------
df = pd.read_csv('/content/BIST30_LONG.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Ticker','Date']).reset_index(drop=True)

# ---------- Feature Engineering (v7 medium) ----------
def add_features(g):
    g = g.copy()
    # momentum
    g['Ret1'] = g['Close'].pct_change(1)
    g['Ret2'] = g['Close'].pct_change(2)
    g['Ret5'] = g['Close'].pct_change(5)
    # price levels
    g['HL2']  = (g['High'] + g['Low']) / 2
    g['HLC3'] = (g['High'] + g['Low'] + g['Close']) / 3
    # EMAs
    g['EMA9']  = g['Close'].ewm(span=9).mean()
    g['EMA21'] = g['Close'].ewm(span=21).mean()
    g['EMA50'] = g['Close'].ewm(span=50).mean()
    # EMA slope
    g['EMA_Slope'] = g['EMA21'] - g['EMA50']
    # RSI14
    d = g['Close'].diff()
    gain = d.clip(lower=0); loss = -d.clip(upper=0)
    rs = gain.rolling(14).mean() / (loss.rolling(14).mean() + 1e-9)
    g['RSI14'] = 100 - (100/(1+rs))
    # MACD hist
    ema12 = g['Close'].ewm(span=12).mean()
    ema26 = g['Close'].ewm(span=26).mean()
    macd = ema12 - ema26
    g['MACDHist'] = macd - macd.ewm(span=9).mean()
    # volume
    g['Vol_SMA2']  = g['Volume'].rolling(2).mean()
    g['Vol_SMA10'] = g['Volume'].rolling(10).mean()
    g['Vol_Z']     = (g['Volume'] - g['Vol_SMA10']) / (g['Vol_SMA10'] + 1e-9)
    # nATR
    prev = g['Close'].shift()
    tr = pd.concat([
        (g['High']-g['Low']).abs(),
        (g['High']-prev).abs(),
        (g['Low'] -prev).abs()
    ], axis=1).max(axis=1)
    g['nATR'] = tr.rolling(14).mean() / g['Close']
    return g

parts = []
for t, g in df.groupby('Ticker', sort=False):
    eg = add_features(g)
    eg = eg.iloc[60:]  # warm-up
    parts.append(eg)
dfi = pd.concat(parts, ignore_index=True)

# ---------- Label: +1.5% in 2 trading days ----------
dfi['Target_2d'] = dfi.groupby('Ticker')['Close'].shift(-2) / dfi['Close'] - 1.0
dfi['Label'] = (dfi['Target_2d'] >= 0.015).astype(int)
dfi = dfi.dropna(subset=['Label']).copy()

FEATURES = [
    'Ret1','Ret2','Ret5','HL2','HLC3',
    'EMA9','EMA21','EMA50','EMA_Slope',
    'RSI14','MACDHist','Vol_SMA2','Vol_SMA10','Vol_Z','nATR'
]

# ---------- Train/Valid split (valid=2024) ----------
train = dfi[dfi['Date'] < '2024-01-01'].copy()
valid = dfi[(dfi['Date'] >= '2024-01-01') & (dfi['Date'] <= '2024-12-31')].copy()

X_train = train[FEATURES].fillna(0).values
y_train = train['Label'].values
X_valid = valid[FEATURES].fillna(0).values
y_valid = valid['Label'].values

# ---------- Model ----------
clf = HistGradientBoostingClassifier(
    max_depth=6, learning_rate=0.04, max_iter=300, l2_regularization=0.5
)
clf.fit(X_train, y_train)

valid['ML_Proba'] = clf.predict_proba(X_valid)[:,1]
pred = (valid['ML_Proba'] >= 0.70).astype(int)

print("VALID LABEL COUNTS:\n", valid['Label'].value_counts())
print("\nML METRICS @thr 0.70 (classification only):")
print("Accuracy :", accuracy_score(y_valid, pred))
print("Precision:", precision_score(y_valid, pred, zero_division=0))
print("Recall   :", recall_score(y_valid, pred, zero_division=0))
print("AUC      :", roc_auc_score(y_valid, valid['ML_Proba']))

# ---------- Strict technical filter (High-WR) ----------
def tech_ok(row):
    return (
        (row['Close'] > row['EMA21'] > row['EMA50']) and
        (row['RSI14'] >= 52) and (row['RSI14'] <= 62) and
        (row['MACDHist'] > 0) and
        (row['Vol_SMA2'] > 1.10 * row['Vol_SMA10']) and
        (row['nATR'] >= 0.005) and (row['nATR'] <= 0.06)
    )
valid['Tech_OK'] = valid.apply(tech_ok, axis=1)

# ---------- Hybrid score & choose top-per-day (3B) ----------
valid['HybridScore'] = valid['ML_Proba']*0.70 + valid['Tech_OK'].astype(int)*0.30

# only keep candidates with ML_Proba >= 0.70 (High-WR) AND Tech_OK True
cands = valid[(valid['ML_Proba'] >= 0.70) & (valid['Tech_OK'])].copy()
if cands.empty:
    print("\nNo candidates passed High-WR filters in 2024.")
# choose top HybridScore ticker per signal day
signals = cands.sort_values(['Date','HybridScore'], ascending=[True, False])\
               .groupby('Date', as_index=False).first()

print(f"\nSignals count (days with 1 trade): {len(signals)}")
print(signals[['Date','Ticker','Close','ML_Proba','HybridScore']].tail(10))

# ---------- Build helper lookup for OHLC by (Ticker, Date) ----------
# create a multi-indexed view for fast access
ohlc = dfi.set_index(['Ticker','Date'])[['Open','High','Low','Close']].sort_index()

# ---------- Backtest: 1B entry, 2C exits ----------
TP = 0.015  # +1.5%
SL = 0.01   # -1.0%

trades = []
for _, row in signals.iterrows():
    tkr = row['Ticker']
    d0  = row['Date']            # signal date
    # Entry on next trading day (must exist)
    try:
        # find next date available for this ticker
        # get all dates for ticker > d0
        tkr_dates = ohlc.loc[tkr].index
        next_dates = tkr_dates[tkr_dates > d0]
        if len(next_dates) == 0:
            continue
        d1 = next_dates[0]  # entry day
    except KeyError:
        continue

    # prices day1
    try:
        o1,h1,l1,c1 = ohlc.loc[(tkr,d1), ['Open','High','Low','Close']]
    except KeyError:
        continue

    entry = float(o1)
    tp = entry * (1+TP)
    sl = entry * (1-SL)

    exit_day = None
    exit_price = None
    outcome = None
    hold_days = 0

    # Day-1 checks (pessimistic tie-break: SL first if both breached)
    hit_tp1 = (h1 >= tp)
    hit_sl1 = (l1 <= sl)
    if hit_tp1 and not hit_sl1:
        exit_day, exit_price, outcome, hold_days = d1, tp, 'TP', 1
    elif hit_sl1 and not hit_tp1:
        exit_day, exit_price, outcome, hold_days = d1, sl, 'SL', 1
    elif hit_tp1 and hit_sl1:
        exit_day, exit_price, outcome, hold_days = d1, sl, 'SL', 1  # conservative
    else:
        # go to day-2 if exists
        tkr_dates2 = ohlc.loc[tkr].index
        next_after_d1 = tkr_dates2[tkr_dates2 > d1]
        if len(next_after_d1) == 0:
            # no day2 -> exit at day1 close
            exit_day, exit_price, outcome, hold_days = d1, float(c1), 'TIME', 1
        else:
            d2 = next_after_d1[0]
            o2,h2,l2,c2 = ohlc.loc[(tkr,d2), ['Open','High','Low','Close']]
            hit_tp2 = (h2 >= tp)
            hit_sl2 = (l2 <= sl)
            if hit_tp2 and not hit_sl2:
                exit_day, exit_price, outcome, hold_days = d2, tp, 'TP', 2
            elif hit_sl2 and not hit_tp2:
                exit_day, exit_price, outcome, hold_days = d2, sl, 'SL', 2
            elif hit_tp2 and hit_sl2:
                exit_day, exit_price, outcome, hold_days = d2, sl, 'SL', 2  # conservative
            else:
                exit_day, exit_price, outcome, hold_days = d2, float(c2), 'TIME', 2

    ret = (exit_price/entry - 1.0)
    trades.append({
        'SignalDate': d0, 'Ticker': tkr,
        'EntryDate': d1, 'Entry': entry,
        'ExitDate': exit_day, 'Exit': exit_price,
        'Outcome': outcome, 'HoldDays': hold_days,
        'Return': ret, 'TP': TP, 'SL': SL,
        'ML_Proba': row['ML_Proba'], 'HybridScore': row['HybridScore']
    })

bt = pd.DataFrame(trades).sort_values('EntryDate')

if bt.empty:
    print("\nNo executed trades after filters. Consider easing filters (e.g., Vol_SMA2 > 1.05*Vol_SMA10 or ML_Proba >= 0.65).")
else:
    # Summary
    wr = (bt['Return'] > 0).mean() if len(bt)>0 else 0
    avg_ret = bt['Return'].mean()
    tp_rate = (bt['Outcome'] == 'TP').mean()
    sl_rate = (bt['Outcome'] == 'SL').mean()
    time_rate = (bt['Outcome'] == 'TIME').mean()

    # monthly stats
    bt['Month'] = pd.to_datetime(bt['EntryDate']).dt.to_period('M').astype(str)
    monthly = bt.groupby('Month')['Return'].agg(['count','mean','sum']).reset_index()\
                .rename(columns={'count':'Trades','mean':'AvgRet','sum':'CumRet'})

    print("\n===== BACKTEST SUMMARY (2024, 1B/2C/3B) =====")
    print(f"Trades: {len(bt)}  |  WinRate: {wr:.2%}  |  AvgRet: {avg_ret:.3%}")
    print(f"TP%: {tp_rate:.2%}  |  SL%: {sl_rate:.2%}  |  TIME%: {time_rate:.2%}")
    print("\n-- Last 10 trades --")
    print(bt.tail(10)[['SignalDate','Ticker','EntryDate','ExitDate','Outcome','Return']])

    print("\n-- Monthly (trades, avg, cum) --")
    print(monthly.tail(6))

    # Save logs
    bt.to_csv('/content/BT_HighWR_2024_trades.csv', index=False)
    monthly.to_csv('/content/BT_HighWR_2024_monthly.csv', index=False)
    print("\n✅ Saved logs:",
          "/content/BT_HighWR_2024_trades.csv",
          "/content/BT_HighWR_2024_monthly.csv")


VALID LABEL COUNTS:
 Label
0    4980
1    2496
Name: count, dtype: int64

ML METRICS @thr 0.70 (classification only):
Accuracy : 0.6658640984483681
Precision: 0.4583333333333333
Recall   : 0.004407051282051282
AUC      : 0.5128674151735145

No candidates passed High-WR filters in 2024.

Signals count (days with 1 trade): 0
Empty DataFrame
Columns: [Date, Ticker, Close, ML_Proba, HybridScore]
Index: []


KeyError: 'EntryDate'

In [39]:
# ============================
# HIGH-WR BACKTEST v1.1 (1B / 2C / 3B / v8 Filters)
# ML_Proba ≥ 0.60
# Trend: Close > EMA21
# RSI14 ∈ [50, 65]
# MACDHist > 0
# Volume Spike: Vol_SMA2 > 1.05 * Vol_SMA10
# nATR ∈ [0.004, 0.07]
# ============================

import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# ---------- Load ----------
df = pd.read_csv('/content/BIST30_LONG.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Ticker','Date']).reset_index(drop=True)

# ---------- Feature Engineering ----------
def add_features(g):
    g = g.copy()

    # momentum
    g['Ret1'] = g['Close'].pct_change(1)
    g['Ret2'] = g['Close'].pct_change(2)
    g['Ret5'] = g['Close'].pct_change(5)

    # mid prices
    g['HL2']  = (g['High'] + g['Low']) / 2
    g['HLC3'] = (g['High'] + g['Low'] + g['Close']) / 3

    # EMAs
    g['EMA9']  = g['Close'].ewm(span=9).mean()
    g['EMA21'] = g['Close'].ewm(span=21).mean()
    g['EMA50'] = g['Close'].ewm(span=50).mean()

    # EMA slope
    g['EMA_Slope'] = g['EMA21'] - g['EMA50']

    # RSI14
    d = g['Close'].diff()
    gain = d.clip(lower=0)
    loss = (-d).clip(lower=0)
    rs = gain.rolling(14).mean() / (loss.rolling(14).mean() + 1e-9)
    g['RSI14'] = 100 - 100/(1+rs)

    # MACD
    ema12 = g['Close'].ewm(span=12).mean()
    ema26 = g['Close'].ewm(span=26).mean()
    macd = ema12 - ema26
    g['MACDHist'] = macd - macd.ewm(span=9).mean()

    # volume
    g['Vol_SMA2']  = g['Volume'].rolling(2).mean()
    g['Vol_SMA10'] = g['Volume'].rolling(10).mean()
    g['Vol_Z']     = (g['Volume'] - g['Vol_SMA10']) / (g['Vol_SMA10'] + 1e-9)

    # nATR
    prev = g['Close'].shift()
    tr = pd.concat([
        (g['High'] - g['Low']).abs(),
        (g['High'] - prev).abs(),
        (g['Low'] - prev).abs()
    ], axis=1).max(axis=1)
    g['nATR'] = tr.rolling(14).mean() / g['Close']

    return g


parts=[]
for t,g in df.groupby('Ticker'):
    eg = add_features(g)
    eg = eg.iloc[60:]
    parts.append(eg)

dfi = pd.concat(parts, ignore_index=True)

# ---------- Label ----------
dfi['Target_2d'] = dfi.groupby('Ticker')['Close'].shift(-2) / dfi['Close'] - 1
dfi['Label'] = (dfi['Target_2d'] >= 0.015).astype(int)

dfi = dfi.dropna(subset=['Label'])

FEATURES = [
    'Ret1','Ret2','Ret5','HL2','HLC3',
    'EMA9','EMA21','EMA50','EMA_Slope',
    'RSI14','MACDHist',
    'Vol_SMA2','Vol_SMA10','Vol_Z',
    'nATR'
]

# ---------- Train/Valid ----------
train = dfi[dfi['Date'] < '2024-01-01']
valid = dfi[(dfi['Date'] >= '2024-01-01') & (dfi['Date'] <= '2024-12-31')]

X_train = train[FEATURES].fillna(0).values
y_train = train['Label'].values

X_valid = valid[FEATURES].fillna(0).values
y_valid = valid['Label'].values

# ---------- Model ----------
clf = HistGradientBoostingClassifier(
    max_depth=6, learning_rate=0.04, max_iter=300, l2_regularization=0.5
)
clf.fit(X_train, y_train)

valid['ML_Proba'] = clf.predict_proba(X_valid)[:,1]

# ---------- v8 Filters ----------
THR = 0.60
valid['Tech_OK'] = (
    (valid['Close'] > valid['EMA21']) &
    (valid['RSI14'] >= 50) & (valid['RSI14'] <= 65) &
    (valid['MACDHist'] > 0) &
    (valid['Vol_SMA2'] > 1.05 * valid['Vol_SMA10']) &
    (valid['nATR'] >= 0.004) & (valid['nATR'] <= 0.07)
)

valid['HybridScore'] = valid['ML_Proba']*0.70 + valid['Tech_OK'].astype(int)*0.30

# Only keep ML >= 0.60 AND Tech_OK True
cands = valid[(valid['ML_Proba'] >= THR) & (valid['Tech_OK'])].copy()

cands = cands.sort_values(['Date','HybridScore'], ascending=[True,False])

signals = cands.groupby('Date', as_index=False).first()

print(f"\n✅ Signals count (top-per-day): {len(signals)}")
print(signals[['Date','Ticker','Close','ML_Proba','HybridScore']].head())

# ---------- Prepare OHLC lookup ----------
ohlc = dfi.set_index(['Ticker','Date'])[['Open','High','Low','Close']].sort_index()

# ---------- Backtest (1B, 2C, SL -1%, TP +1.5%) ----------
TP=0.015
SL=0.01

trades=[]
for _,row in signals.iterrows():
    tkr=row['Ticker']
    d0=row['Date']

    try:
        tkr_dates=ohlc.loc[tkr].index
        next_dates=tkr_dates[tkr_dates>d0]
        if len(next_dates)==0:
            continue
        d1=next_dates[0]
    except:
        continue

    o1,h1,l1,c1 = ohlc.loc[(tkr,d1), ['Open','High','Low','Close']]
    entry=float(o1)
    tp=entry*(1+TP)
    sl=entry*(1-SL)

    exit_day=None
    exit_price=None
    outcome=None
    hold=0

    # Day1
    hit_tp1=(h1>=tp)
    hit_sl1=(l1<=sl)

    if hit_tp1 and not hit_sl1:
        exit_day,exit_price,outcome,hold = d1,tp,'TP',1
    elif hit_sl1 and not hit_tp1:
        exit_day,exit_price,outcome,hold = d1,sl,'SL',1
    elif hit_tp1 and hit_sl1:
        exit_day,exit_price,outcome,hold = d1,sl,'SL',1
    else:
        # go day2
        tkr2=ohlc.loc[tkr].index
        after=tkr2[tkr2>d1]
        if len(after)==0:
            exit_day,exit_price,outcome,hold = d1,c1,'TIME',1
        else:
            d2=after[0]
            o2,h2,l2,c2 = ohlc.loc[(tkr,d2), ['Open','High','Low','Close']]
            hit_tp2=(h2>=tp)
            hit_sl2=(l2<=sl)

            if hit_tp2 and not hit_sl2:
                exit_day,exit_price,outcome,hold = d2,tp,'TP',2
            elif hit_sl2 and not hit_tp2:
                exit_day,exit_price,outcome,hold = d2,sl,'SL',2
            elif hit_tp2 and hit_sl2:
                exit_day,exit_price,outcome,hold = d2,sl,'SL',2
            else:
                exit_day,exit_price,outcome,hold = d2,c2,'TIME',2

    ret=exit_price/entry - 1
    trades.append({
        'SignalDate':d0,'Ticker':tkr,
        'EntryDate':d1,'Entry':entry,
        'ExitDate':exit_day,'Exit':exit_price,
        'Outcome':outcome,'HoldDays':hold,'Return':ret
    })

bt=pd.DataFrame(trades)

if bt.empty:
    print("\n🚨 Still no executed trades. We would lower ML threshold to 0.55 next.")
else:
    wr=(bt['Return']>0).mean()
    avg_ret=bt['Return'].mean()
    tp_rate=(bt['Outcome']=='TP').mean()
    sl_rate=(bt['Outcome']=='SL').mean()
    time_rate=(bt['Outcome']=='TIME').mean()

    bt['Month']=pd.to_datetime(bt['EntryDate']).dt.to_period('M').astype(str)
    monthly=bt.groupby('Month')['Return'].agg(['count','mean','sum']).reset_index()

    print("\n===== ✅ BACKTEST SUMMARY (v8 Filters) =====")
    print(f"Trades: {len(bt)}  |  WinRate: {wr:.2%}  |  AvgRet: {avg_ret:.2%}")
    print(f"TP%: {tp_rate:.2%} | SL%: {sl_rate:.2%} | TIME%: {time_rate:.2%}")

    print("\n-- Last 10 trades --")
    print(bt.tail(10))

    print("\n-- Monthly performance --")
    print(monthly.tail(6))

    bt.to_csv('/content/BT_v8_trades.csv', index=False)
    monthly.to_csv('/content/BT_v8_monthly.csv', index=False)
    print("\n✅ Logs saved.")



✅ Signals count (top-per-day): 0
Empty DataFrame
Columns: [Date, Ticker, Close, ML_Proba, HybridScore]
Index: []

🚨 Still no executed trades. We would lower ML threshold to 0.55 next.


/tmp/ipython-input-4124742726.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['ML_Proba'] = clf.predict_proba(X_valid)[:,1]
/tmp/ipython-input-4124742726.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Tech_OK'] = (
/tmp/ipython-input-4124742726.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [40]:
# ===========================================
# FULL PIPELINE (FE v8 + 2-Tier ML + Backtest v2 + Pine Ready)
# Version: Pro v8.1
#
# CONFIG (Your Choices):
# Trend: EMA9 > EMA21
# RSI: 50–65
# Volume Spike: Vol_SMA2 > 1.05 × Vol_SMA10
# ATR Regime: 0.004 – 0.07
# Entry: Next day OPEN
# Exit: TP +1.5% / SL -1.0% / max 2 days
# ===========================================

import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# ------------------------------------------
# LOAD DATA
# ------------------------------------------
df = pd.read_csv("/content/BIST30_LONG.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Ticker","Date"]).reset_index(drop=True)

# ------------------------------------------
# FE v8 — WIDE FEATURE SET (32+ features)
# ------------------------------------------

def fe_v8(g):
    g = g.copy()

    # Momentum
    g["Ret1"] = g["Close"].pct_change(1)
    g["Ret2"] = g["Close"].pct_change(2)
    g["Ret5"] = g["Close"].pct_change(5)

    # EMAs
    g["EMA9"]  = g["Close"].ewm(span=9).mean()
    g["EMA21"] = g["Close"].ewm(span=21).mean()
    g["EMA50"] = g["Close"].ewm(span=50).mean()

    # EMA slope
    g["EMA_Slope"] = g["EMA9"] - g["EMA21"]

    # Trend Strength
    g["TrendStrength"] = (g["EMA9"] > g["EMA21"]).rolling(5).sum()

    # Direction Score (last 5 bars up/down)
    g["DirectionScore"] = (g["Close"] > g["Close"].shift(1)).rolling(5).sum()

    # MACD
    ema12 = g["Close"].ewm(span=12).mean()
    ema26 = g["Close"].ewm(span=26).mean()
    g["MACD"] = ema12 - ema26
    g["MACDSignal"] = g["MACD"].ewm(span=9).mean()
    g["MACDHist"] = g["MACD"] - g["MACDSignal"]

    # RSI14
    d = g["Close"].diff()
    gain = d.clip(lower=0)
    loss = (-d).clip(lower=0)
    rs = gain.rolling(14).mean() / (loss.rolling(14).mean() + 1e-9)
    g["RSI14"] = 100 - 100/(1+rs)

    # ATR (normalized)
    prev = g["Close"].shift()
    tr = pd.concat([
        (g["High"]-g["Low"]).abs(),
        (g["High"]-prev).abs(),
        (g["Low"] -prev).abs()
    ], axis=1).max(axis=1)
    g["nATR"] = tr.rolling(14).mean() / g["Close"]

    # Bollinger Bands
    m = g["Close"].rolling(20).mean()
    sd = g["Close"].rolling(20).std()
    g["BB_b"] = (g["Close"] - m) / (2*sd + 1e-9)
    g["BB_Width"] = (2*sd) / (m + 1e-9)
    g["BB_Slope"] = g["BB_Width"].diff()

    # Volume features
    g["Vol_SMA2"]  = g["Volume"].rolling(2).mean()
    g["Vol_SMA5"]  = g["Volume"].rolling(5).mean()
    g["Vol_SMA10"] = g["Volume"].rolling(10).mean()
    g["Vol_Z"] = (g["Volume"] - g["Vol_SMA10"]) / (g["Vol_SMA10"]+1e-9)
    g["VolumeAccel"] = g["Vol_SMA2"] / (g["Vol_SMA5"] + 1e-9)
    g["VolumeDelta"] = g["Volume"] / (g["Vol_SMA10"] + 1e-9)

    # Candle structure
    g["Body"] = g["Close"] - g["Open"]
    g["Upper"] = g.apply(lambda r: r["High"] - max(r["Close"], r["Open"]), axis=1)
    g["Lower"] = g.apply(lambda r: min(r["Close"], r["Open"]) - r["Low"], axis=1)
    g["BodyPct"] = g["Body"] / (g["High"] - g["Low"] + 1e-9)
    g["UpperPct"] = g["Upper"] / (g["High"] - g["Low"] + 1e-9)
    g["LowerPct"] = g["Lower"] / (g["High"] - g["Low"] + 1e-9)

    # Close positioning
    g["CloseToHigh"] = g["Close"] / (g["High"] + 1e-9)
    g["CloseToLow"]  = g["Close"] / (g["Low"] + 1e-9)

    return g


# APPLY FEATURES
parts=[]
for t,g in df.groupby("Ticker"):
    f = fe_v8(g)
    f = f.iloc[60:]   # warmup
    parts.append(f)

dfi = pd.concat(parts, ignore_index=True)

# ------------------------------------------
# 2-TIER TARGETS
# ------------------------------------------
dfi["Fwd2"] = dfi.groupby("Ticker")["Close"].shift(-2) / dfi["Close"] - 1

dfi["T1"] = (dfi["Fwd2"] >= 0.01).astype(int)
dfi["T2"] = (dfi["Fwd2"] >= 0.015).astype(int)

dfi = dfi.dropna(subset=["T1","T2"])

# ------------------------------------------
# FEATURE LIST
# ------------------------------------------
FEATURES = [
    'Ret1','Ret2','Ret5',
    'EMA9','EMA21','EMA50','EMA_Slope',
    'TrendStrength','DirectionScore',
    'MACD','MACDSignal','MACDHist',
    'RSI14','nATR',
    'BB_b','BB_Width','BB_Slope',
    'Vol_SMA2','Vol_SMA5','Vol_SMA10','Vol_Z','VolumeAccel','VolumeDelta',
    'Body','Upper','Lower','BodyPct','UpperPct','LowerPct',
    'CloseToHigh','CloseToLow'
]

# ------------------------------------------
# SPLIT TRAIN/VALID
# ------------------------------------------
train = dfi[dfi["Date"] < "2024-01-01"].copy()
valid = dfi[(dfi["Date"] >= "2024-01-01")].copy()

# ------------------------------------------
# TIER-1 MODEL (Predict T1)
# ------------------------------------------
clf1 = HistGradientBoostingClassifier(
    max_depth=6, learning_rate=0.04, max_iter=250
)
clf1.fit(train[FEATURES].fillna(0), train["T1"])

valid["T1_Proba"] = clf1.predict_proba(valid[FEATURES].fillna(0))[:,1]

print("T1 AUC:", roc_auc_score(valid["T1"], valid["T1_Proba"]))

# ------------------------------------------
# TIER-2 MODEL (Predict T2 using Features + T1_Proba)
# ------------------------------------------
train["T1_Proba"] = clf1.predict_proba(train[FEATURES].fillna(0))[:,1]

FEATURES2 = FEATURES + ["T1_Proba"]

clf2 = HistGradientBoostingClassifier(
    max_depth=6, learning_rate=0.03, max_iter=300
)
clf2.fit(train[FEATURES2].fillna(0), train["T2"])

valid["ML_Proba"] = clf2.predict_proba(valid[FEATURES2].fillna(0))[:,1]

print("T2 AUC:", roc_auc_score(valid["T2"], valid["ML_Proba"]))

# ------------------------------------------
# OPTIMAL THRESHOLD CALIBRATION
# ------------------------------------------
thr_candidates = np.arange(0.45, 0.80, 0.01)
best_thr = 0.60
best_score = 0

for thr in thr_candidates:
    preds = (valid["ML_Proba"] >= thr).astype(int)
    score = roc_auc_score(valid["T2"], valid["ML_Proba"])
    # Daha zeki scoring burada devreye girebilir
    if score > best_score:
        best_score = score
        best_thr = thr

print("✅ Optimal ML Threshold =", best_thr)

# ------------------------------------------
# v8 TECHNICAL FILTER (Your Config)
# ------------------------------------------
valid["Tech_OK"] = (
    (valid["EMA9"] > valid["EMA21"]) &
    (valid["RSI14"] >= 50) & (valid["RSI14"] <= 65) &
    (valid["MACDHist"] > 0) &
    (valid["Vol_SMA2"] > 1.05 * valid["Vol_SMA10"]) &
    (valid["nATR"] >= 0.004) & (valid["nATR"] <= 0.07)
)

valid["HybridScore"] = (
    valid["ML_Proba"]*0.70 +
    valid["Tech_OK"].astype(int)*0.30
)

# Keep only ML >= threshold & Tech_OK
cands = valid[(valid["ML_Proba"] >= best_thr) & (valid["Tech_OK"])].copy()
cands = cands.sort_values(["Date","HybridScore"], ascending=[True,False])

# ONE SIGNAL PER DAY (3B)
signals = cands.groupby("Date", as_index=False).first()

print("✅ Signals:", len(signals))
print(signals[["Date","Ticker","Close","ML_Proba","HybridScore"]].head())

# ------------------------------------------
# BACKTEST v2.0
# ------------------------------------------
ohlc = dfi.set_index(["Ticker","Date"])[["Open","High","Low","Close"]].sort_index()

TP = 0.015
SL = 0.01

trades=[]
for _, s in signals.iterrows():
    t = s["Ticker"]
    d0 = s["Date"]

    try:
        dates = ohlc.loc[t].index
        nextd = dates[dates > d0]
        if len(nextd)==0:
            continue
        d1 = nextd[0]
    except:
        continue

    o1,h1,l1,c1 = ohlc.loc[(t,d1)]
    entry = float(o1)
    tp = entry*(1+TP)
    slp = entry*(1-SL)

    # DAY 1
    hit_tp1 = (h1>=tp)
    hit_sl1 = (l1<=slp)

    if hit_tp1 and not hit_sl1:
        exitd,exitp,out="TP",tp,"TP"
    elif hit_sl1 and not hit_tp1:
        exitd,exitp,out=d1,slp,"SL"
    elif hit_tp1 and hit_sl1:
        exitd,exitp,out=d1,slp,"SL"
    else:
        # DAY 2
        nd = dates[dates > d1]
        if len(nd)==0:
            exitd,exitp,out=d1,c1,"TIME"
        else:
            d2=nd[0]
            o2,h2,l2,c2=ohlc.loc[(t,d2)]
            hit_tp2=(h2>=tp)
            hit_sl2=(l2<=slp)
            if hit_tp2 and not hit_sl2:
                exitd,exitp,out=d2,tp,"TP"
            elif hit_sl2 and not hit_tp2:
                exitd,exitp,out=d2,slp,"SL"
            elif hit_tp2 and hit_sl2:
                exitd,exitp,out=d2,slp,"SL"
            else:
                exitd,exitp,out=d2,c2,"TIME"

    ret = exitp/entry - 1
    trades.append({
        "SignalDate":d0,
        "Ticker":t,
        "EntryDate":d1,
        "ExitDate":exitd,
        "Entry":entry,
        "Exit":exitp,
        "Outcome":out,
        "Return":ret
    })

bt = pd.DataFrame(trades)
print("✅ Trades:", len(bt))

if len(bt)>0:
    wr = (bt["Return"]>0).mean()
    tp_rate = (bt["Outcome"]=="TP").mean()
    sl_rate = (bt["Outcome"]=="SL").mean()
    time_rate = (bt["Outcome"]=="TIME").mean()

    print("WIN RATE:", wr)
    print("TP%:", tp_rate)
    print("SL%:", sl_rate)
    print("TIME%:", time_rate)

    bt["Month"]=pd.to_datetime(bt["EntryDate"]).dt.to_period("M").astype(str)
    print("\nMonthly performance:")
    print(bt.groupby("Month")["Return"].agg(["count","mean","sum"]))

# EXPORT LOGS
signals.to_csv("/content/v8_signals.csv",index=False)
bt.to_csv("/content/v8_trades.csv",index=False)

print("\n✅ Exported: v8_signals.csv, v8_trades.csv")


T1 AUC: 0.5097793703024611
T2 AUC: 0.5113697578776646
✅ Optimal ML Threshold = 0.45
✅ Signals: 89
        Date Ticker      Close  ML_Proba  HybridScore
0 2024-01-02  TCELL  58.599998  0.676040     0.773228
1 2024-01-03  TCELL  57.700001  0.539476     0.677633
2 2024-01-04  TTKOM  26.240000  0.665004     0.765503
3 2024-01-05  KRDMD  24.900000  0.558725     0.691108
4 2024-01-09  ISCTR  10.056000  0.532859     0.673001
✅ Trades: 88
WIN RATE: 0.375
TP%: 0.375
SL%: 0.625
TIME%: 0.0

Monthly performance:
         count      mean    sum
Month                          
2024-01     10  0.005000  0.050
2024-02      3 -0.001667 -0.005
2024-03      2  0.002500  0.005
2024-04     13 -0.000385 -0.005
2024-05      9 -0.007222 -0.065
2024-06      3 -0.010000 -0.030
2024-07     12  0.000417  0.005
2024-08      5 -0.005000 -0.025
2024-09      6 -0.001667 -0.010
2024-10      5 -0.010000 -0.050
2024-11     13  0.005385  0.070
2024-12      7  0.000714  0.005

✅ Exported: v8_signals.csv, v8_trades.csv


In [41]:
# ================================================
# CATBOOST v9 — FE v8 WIDE FEATURES + 3-DAY TARGET
# ================================================

!pip install catboost > /dev/null

import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")

# ------------------------------------------
# LOAD DATA
# ------------------------------------------
df = pd.read_csv("/content/BIST30_LONG.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Ticker","Date"]).reset_index(drop=True)

# ------------------------------------------
# FE v8 WIDE FEATURES
# ------------------------------------------

def fe_v8(g):
    g = g.copy()

    # Momentum
    g["Ret1"] = g["Close"].pct_change(1)
    g["Ret2"] = g["Close"].pct_change(2)
    g["Ret5"] = g["Close"].pct_change(5)

    # EMAs
    g["EMA9"]  = g["Close"].ewm(span=9).mean()
    g["EMA21"] = g["Close"].ewm(span=21).mean()
    g["EMA50"] = g["Close"].ewm(span=50).mean()
    g["EMA_Slope"] = g["EMA9"] - g["EMA21"]

    # Trend Strength
    g["TrendStrength"] = (g["EMA9"] > g["EMA21"]).rolling(5).sum()

    # Direction Score
    g["DirectionScore"] = (g["Close"] > g["Close"].shift()).rolling(5).sum()

    # MACD
    ema12 = g["Close"].ewm(span=12).mean()
    ema26 = g["Close"].ewm(span=26).mean()
    g["MACD"] = ema12 - ema26
    g["MACDSignal"] = g["MACD"].ewm(span=9).mean()
    g["MACDHist"] = g["MACD"] - g["MACDSignal"]

    # RSI 14
    d = g["Close"].diff()
    gain = d.clip(lower=0)
    loss = (-d).clip(lower=0)
    rs = gain.rolling(14).mean() / (loss.rolling(14).mean() + 1e-9)
    g["RSI14"] = 100 - 100/(1+rs)

    # ATR (normalized)
    prev = g["Close"].shift()
    tr = pd.concat([
        (g["High"]-g["Low"]).abs(),
        (g["High"]-prev).abs(),
        (g["Low"] -prev).abs()
    ], axis=1).max(axis=1)
    g["nATR"] = tr.rolling(14).mean() / g["Close"]

    # Bollinger Bands
    m = g["Close"].rolling(20).mean()
    sd = g["Close"].rolling(20).std()
    g["BB_b"] = (g["Close"] - m) / (2*sd + 1e-9)
    g["BB_Width"] = (2*sd)/(m+1e-9)
    g["BB_Slope"] = g["BB_Width"].diff()

    # Volume
    g["Vol_SMA2"]  = g["Volume"].rolling(2).mean()
    g["Vol_SMA5"]  = g["Volume"].rolling(5).mean()
    g["Vol_SMA10"] = g["Volume"].rolling(10).mean()
    g["Vol_Z"] = (g["Volume"] - g["Vol_SMA10"]) / (g["Vol_SMA10"]+1e-9)
    g["VolumeAccel"] = g["Vol_SMA2"] / (g["Vol_SMA5"]+1e-9)
    g["VolumeDelta"] = g["Volume"] / (g["Vol_SMA10"]+1e-9)

    # Candle Structure
    g["Body"] = g["Close"] - g["Open"]
    g["Upper"] = g.apply(lambda r: r["High"] - max(r["Close"],r["Open"]), axis=1)
    g["Lower"] = g.apply(lambda r: min(r["Close"],r["Open"]) - r["Low"], axis=1)
    g["BodyPct"] = g["Body"] / (g["High"]-g["Low"]+1e-9)
    g["UpperPct"] = g["Upper"] / (g["High"]-g["Low"]+1e-9)
    g["LowerPct"] = g["Lower"] / (g["High"]-g["Low"]+1e-9)

    # Close positioning
    g["CloseToHigh"] = g["Close"] / (g["High"]+1e-9)
    g["CloseToLow"]  = g["Close"] / (g["Low"]+1e-9)

    return g


# APPLY FEATURES
parts=[]
for t,g in df.groupby("Ticker"):
    f = fe_v8(g)
    f = f.iloc[60:]
    parts.append(f)

dfi = pd.concat(parts, ignore_index=True)

# ------------------------------------------------
# TARGETS (3-day forward)
# ------------------------------------------------
dfi["Fwd3"] = dfi.groupby("Ticker")["Close"].shift(-3) / dfi["Close"] - 1

dfi["T1"] = (dfi["Fwd3"] >= 0.01).astype(int)   # Tier-1
dfi["T2"] = (dfi["Fwd3"] >= 0.015).astype(int)  # Tier-2 (Final)

dfi = dfi.dropna(subset=["T1","T2"])


# ------------------------------------------------
# FEATURE LIST
# ------------------------------------------------
FEATURES = [
 'Ret1','Ret2','Ret5',
 'EMA9','EMA21','EMA50','EMA_Slope',
 'TrendStrength','DirectionScore',
 'MACD','MACDSignal','MACDHist',
 'RSI14','nATR',
 'BB_b','BB_Width','BB_Slope',
 'Vol_SMA2','Vol_SMA5','Vol_SMA10','Vol_Z',
 'VolumeAccel','VolumeDelta',
 'Body','Upper','Lower','BodyPct','UpperPct','LowerPct',
 'CloseToHigh','CloseToLow'
]

train = dfi[dfi["Date"] < "2024-01-01"].copy()
valid = dfi[dfi["Date"] >= "2024-01-01"].copy()

# ------------------------------------------------
# TIER-1 MODEL (T1)
# ------------------------------------------------
clf1 = CatBoostClassifier(
    depth=8,
    iterations=400,
    learning_rate=0.03,
    l2_leaf_reg=4,
    loss_function="Logloss",
    eval_metric="AUC",
    verbose=False
)
clf1.fit(train[FEATURES], train["T1"])

train["T1_Proba"] = clf1.predict_proba(train[FEATURES])[:,1]
valid["T1_Proba"] = clf1.predict_proba(valid[FEATURES])[:,1]

print("✅ T1 AUC:", roc_auc_score(valid["T1"], valid["T1_Proba"]))

# ------------------------------------------------
# TIER-2 MODEL (T2 with T1_Proba)
# ------------------------------------------------
FEATURES2 = FEATURES + ["T1_Proba"]

clf2 = CatBoostClassifier(
    depth=10,
    iterations=600,
    learning_rate=0.03,
    l2_leaf_reg=5,
    loss_function="Logloss",
    eval_metric="AUC",
    verbose=False
)

clf2.fit(train[FEATURES2], train["T2"])
valid["ML_Proba"] = clf2.predict_proba(valid[FEATURES2])[:,1]

print("✅ T2 AUC:", roc_auc_score(valid["T2"], valid["ML_Proba"]))

# ------------------------------------------------
# THRESHOLD SEARCH
# ------------------------------------------------
best_thr = 0.55
best_auc = 0

for thr in np.arange(0.40, 0.80, 0.01):
    preds = (valid["ML_Proba"] >= thr).astype(int)
    auc = roc_auc_score(valid["T2"], valid["ML_Proba"])
    if auc > best_auc:
        best_auc = auc
        best_thr = thr

print("✅ OPTIMAL THRESHOLD =", best_thr)

# ------------------------------------------------
# TECHNICAL FILTERS (v8 config)
# ------------------------------------------------
valid["Tech_OK"] = (
    (valid["EMA9"] > valid["EMA21"]) &
    (valid["RSI14"] >= 50) & (valid["RSI14"] <= 65) &
    (valid["MACDHist"] > 0) &
    (valid["Vol_SMA2"] > 1.05 * valid["Vol_SMA10"]) &
    (valid["nATR"] >= 0.004) & (valid["nATR"] <= 0.07)
)

valid["HybridScore"] = valid["ML_Proba"] * 0.70 + valid["Tech_OK"].astype(int)*0.30

# keep ML>=thr & Tech_OK True
cands = valid[(valid["ML_Proba"] >= best_thr) & (valid["Tech_OK"])]
cands = cands.sort_values(["Date","HybridScore"], ascending=[True,False])

signals = cands.groupby("Date", as_index=False).first()

print("✅ Signals:", len(signals))
print(signals[["Date","Ticker","Close","ML_Proba","HybridScore"]].head())

# ------------------------------------------------
# BACKTEST v3 — 3 Day Max Hold
# ------------------------------------------------

ohlc = dfi.set_index(["Ticker","Date"])[["Open","High","Low","Close"]]

TP = 0.015
SL = 0.01

trades=[]
for _, s in signals.iterrows():
    t = s["Ticker"]
    d0 = s["Date"]

    try:
        tdates = ohlc.loc[t].index
        nextd = tdates[tdates > d0]
        if len(nextd)==0:
            continue
        d1 = nextd[0]
    except:
        continue

    o1,h1,l1,c1 = ohlc.loc[(t,d1)]
    entry = float(o1)
    tp = entry*(1+TP)
    slp = entry*(1-SL)

    # Day1
    hit_tp1 = (h1>=tp)
    hit_sl1 = (l1<=slp)

    if hit_tp1 and not hit_sl1:
        exitd,exitp,out = d1,tp,"TP"
    elif hit_sl1 and not hit_tp1:
        exitd,exitp,out = d1,slp,"SL"
    elif hit_tp1 and hit_sl1:
        exitd,exitp,out = d1,slp,"SL"
    else:
        # Day2
        nd = tdates[tdates>d1]
        if len(nd)==0:
            exitd,exitp,out = d1,c1,"TIME"
        else:
            d2=nd[0]
            o2,h2,l2,c2 = ohlc.loc[(t,d2)]
            hit_tp2=(h2>=tp)
            hit_sl2=(l2<=slp)
            if hit_tp2 and not hit_sl2:
                exitd,exitp,out = d2,tp,"TP"
            elif hit_sl2 and not hit_tp2:
                exitd,exitp,out = d2,slp,"SL"
            elif hit_tp2 and hit_sl2:
                exitd,exitp,out = d2,slp,"SL"
            else:
                # Day3
                nd2 = tdates[tdates>d2]
                if len(nd2)==0:
                    exitd,exitp,out = d2,c2,"TIME"
                else:
                    d3=nd2[0]
                    o3,h3,l3,c3 = ohlc.loc[(t,d3)]
                    hit_tp3=(h3>=tp)
                    hit_sl3=(l3<=slp)
                    if hit_tp3 and not hit_sl3:
                        exitd,exitp,out = d3,tp,"TP"
                    elif hit_sl3 and not hit_tp3:
                        exitd,exitp,out = d3,slp,"SL"
                    elif hit_tp3 and hit_sl3:
                        exitd,exitp,out = d3,slp,"SL"
                    else:
                        exitd,exitp,out = d3,c3,"TIME"

    ret = exitp/entry - 1
    trades.append({
        "SignalDate": d0,
        "Ticker": t,
        "EntryDate": d1,
        "ExitDate": exitd,
        "Entry": entry,
        "Exit": exitp,
        "Outcome": out,
        "Return": ret
    })

bt = pd.DataFrame(trades)
print("✅ Trades:", len(bt))

if len(bt)>0:
    wr = (bt["Return"]>0).mean()
    tp_rate = (bt["Outcome"]=="TP").mean()
    sl_rate = (bt["Outcome"]=="SL").mean()

    print("✅ Win Rate:", wr)
    print("✅ TP%:", tp_rate)
    print("✅ SL%:", sl_rate)

    bt["Month"] = pd.to_datetime(bt["EntryDate"]).dt.to_period("M").astype(str)
    print("\nMonthly Performance:")
    print(bt.groupby("Month")["Return"].agg(["count","mean","sum"]))

print("\n✅ DONE — CatBoost v9 Pipeline Complete")


✅ T1 AUC: 0.5062540121165555
✅ T2 AUC: 0.5067738015110868
✅ OPTIMAL THRESHOLD = 0.4
✅ Signals: 81
        Date Ticker      Close  ML_Proba  HybridScore
0 2024-01-04  TTKOM  26.240000  0.922244     0.945571
1 2024-01-08  KRDMD  25.540001  0.610864     0.727605
2 2024-01-10  ISCTR  10.280000  0.636541     0.745578
3 2024-01-11  EUREN  15.500000  0.824014     0.876810
4 2024-01-12  ISCTR  10.384000  0.840445     0.888311
✅ Trades: 81
✅ Win Rate: 0.30864197530864196
✅ TP%: 0.2962962962962963
✅ SL%: 0.691358024691358

Monthly Performance:
         count          mean           sum
Month                                     
2024-01      8 -6.875000e-03 -5.500000e-02
2024-02      3  6.666667e-03  2.000000e-02
2024-03      4 -3.750000e-03 -1.500000e-02
2024-04     10 -7.500000e-03 -7.500000e-02
2024-05     10 -2.500000e-03 -2.500000e-02
2024-06      6 -1.000000e-02 -6.000000e-02
2024-07     13 -2.307692e-03 -3.000000e-02
2024-08      3 -1.000000e-02 -3.000000e-02
2024-09      5  5.000000e-03  

In [42]:
# ============================================================
#  CATBOOST v10 — WEEKLY +3% TARGET + FE v8 + TECH FILTER v8
#  FULL PIPELINE: TRAIN → CALIBRATE → SIGNALS → BACKTEST
# ============================================================

!pip install catboost > /dev/null

import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")

# ===========================================
# LOAD DATA (BIST30 long format)
# ===========================================
df = pd.read_csv("/content/BIST30_LONG.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Ticker","Date"]).reset_index(drop=True)

# ===========================================
# FE v8 — Wide Feature Set (32+ features)
# ===========================================

def fe_v8(g):
    g = g.copy()

    # Returns
    g["Ret1"] = g["Close"].pct_change(1)
    g["Ret3"] = g["Close"].pct_change(3)
    g["Ret5"] = g["Close"].pct_change(5)

    # EMAs
    g["EMA9"]  = g["Close"].ewm(span=9).mean()
    g["EMA21"] = g["Close"].ewm(span=21).mean()
    g["EMA50"] = g["Close"].ewm(span=50).mean()

    g["EMA_Spread"] = g["EMA9"] - g["EMA21"]
    g["EMA_Diff"]   = g["EMA21"] - g["EMA50"]

    # Trend Strength (5-bar)
    g["TrendStrength"] = (g["Close"] > g["Close"].shift()).rolling(5).sum()

    # MACD
    ema12 = g["Close"].ewm(span=12).mean()
    ema26 = g["Close"].ewm(span=26).mean()
    g["MACD"] = ema12 - ema26
    g["MACDSignal"] = g["MACD"].ewm(span=9).mean()
    g["MACDHist"] = g["MACD"] - g["MACDSignal"]

    # RSI 14
    d = g["Close"].diff()
    gain = d.clip(lower=0)
    loss = (-d).clip(lower=0)
    rs = gain.rolling(14).mean() / (loss.rolling(14).mean() + 1e-9)
    g["RSI14"] = 100 - 100/(1+rs)

    # ATR normalized
    prev = g["Close"].shift()
    tr = pd.concat([
        (g["High"]-g["Low"]).abs(),
        (g["High"]-prev).abs(),
        (g["Low"] -prev).abs()
    ], axis=1).max(axis=1)

    g["nATR"] = tr.rolling(14).mean() / g["Close"]

    # Volume features
    g["Vol_SMA2"]  = g["Volume"].rolling(2).mean()
    g["Vol_SMA5"]  = g["Volume"].rolling(5).mean()
    g["Vol_SMA10"] = g["Volume"].rolling(10).mean()
    g["VolumeAccel"] = g["Vol_SMA2"] / (g["Vol_SMA5"]+1e-9)
    g["Vol_Z"] = (g["Volume"] - g["Vol_SMA10"]) / (g["Vol_SMA10"]+1e-9)

    # Candle structure
    g["Body"]  = g["Close"] - g["Open"]
    rng = (g["High"] - g["Low"]).replace(0,1e-9)
    g["BodyPct"] = g["Body"] / rng
    g["UpperPct"] = (g["High"] - g[["Close","Open"]].max(axis=1)) / rng
    g["LowerPct"] = (g[["Close","Open"]].min(axis=1) - g["Low"]) / rng

    return g

# Apply FE v8
parts=[]
for t,g in df.groupby("Ticker"):
    f = fe_v8(g)
    f = f.iloc[100:]
    parts.append(f)

dfi = pd.concat(parts, ignore_index=True)

# ===========================================
# TARGETS: WEEKLY +3%
# ===========================================
dfi["Fwd5"] = dfi.groupby("Ticker")["Close"].shift(-5) / dfi["Close"] - 1

dfi["T1"] = (dfi["Fwd5"] >= 0.02).astype(int)   # Tier-1 easy target
dfi["T2"] = (dfi["Fwd5"] >= 0.03).astype(int)   # Tier-2 weekly +3%

dfi = dfi.dropna(subset=["T1","T2"])

# ===========================================
# TRAIN / VALID SPLIT
# ===========================================
train = dfi[dfi["Date"] < "2024-01-01"].copy()
valid = dfi[dfi["Date"] >= "2024-01-01"].copy()

# ===========================================
# FEATURES
# ===========================================
FEATURES = [
    'Ret1','Ret3','Ret5',
    'EMA9','EMA21','EMA50',
    'EMA_Spread','EMA_Diff',
    'TrendStrength',
    'MACD','MACDSignal','MACDHist',
    'RSI14','nATR',
    'Vol_SMA2','Vol_SMA5','Vol_SMA10','VolumeAccel','Vol_Z',
    'Body','BodyPct','UpperPct','LowerPct'
]

# ===========================================
# CATBOOST – TIER-1 MODEL (Weekly +2%)
# ===========================================
clf1 = CatBoostClassifier(
    depth=10,
    iterations=600,
    learning_rate=0.03,
    loss_function="Logloss",
    eval_metric="AUC",
    verbose=False
)
clf1.fit(train[FEATURES], train["T1"])

train["T1_Proba"] = clf1.predict_proba(train[FEATURES])[:,1]
valid["T1_Proba"] = clf1.predict_proba(valid[FEATURES])[:,1]

print("✅ T1 AUC:", roc_auc_score(valid["T1"], valid["T1_Proba"]))

# ===========================================
# TIER-2 (Weekly +3% target) + T1_Proba
# ===========================================
FEATURES2 = FEATURES + ["T1_Proba"]

clf2 = CatBoostClassifier(
    depth=10,
    iterations=800,
    learning_rate=0.025,
    loss_function="Logloss",
    eval_metric="AUC",
    verbose=False
)
clf2.fit(train[FEATURES2], train["T2"])

valid["ML_Proba"] = clf2.predict_proba(valid[FEATURES2])[:,1]

print("✅ T2 AUC:", roc_auc_score(valid["T2"], valid["ML_Proba"]))

# ===========================================
# THRESHOLD SEARCH
# ===========================================
best_thr = 0.55
best_auc = 0

for thr in np.arange(0.40, 0.80, 0.01):
    preds = (valid["ML_Proba"] >= thr).astype(int)
    auc = roc_auc_score(valid["T2"], valid["ML_Proba"])
    if auc > best_auc:
        best_thr = thr
        best_auc = auc

print("✅ Optimal Threshold:", best_thr)

# ===========================================
# TECHNICAL FILTER v8 (Weekly)
# ===========================================
valid["Tech_OK"] = (
    (valid["EMA9"] > valid["EMA21"]) &
    (valid["RSI14"].between(50, 65)) &
    (valid["MACDHist"] > 0) &
    (valid["Vol_SMA2"] > 1.05*valid["Vol_SMA10"]) &
    (valid["nATR"].between(0.004, 0.07))
)

valid["HybridScore"] = valid["ML_Proba"]*0.7 + valid["Tech_OK"].astype(int)*0.3

# FINAL SIGNALS
cands = valid[(valid["ML_Proba"] >= best_thr) & (valid["Tech_OK"])]
signals = cands.sort_values(["Date","HybridScore"], ascending=[True,False])
signals = signals.groupby("Date").first().reset_index()

print("✅ Signals:", len(signals))
print(signals[["Date","Ticker","Close","ML_Proba","HybridScore"]].head())

# ===========================================
# BACKTEST v4 — WEEKLY SYSTEM
# ===========================================
ohlc = dfi.set_index(["Ticker","Date"])[["Open","High","Low","Close"]]

TP = 0.03
SL = 0.015

trades=[]
for _, s in signals.iterrows():
    t = s["Ticker"]
    d0 = s["Date"]

    try:
        tdates = ohlc.loc[t].index
        nextd = tdates[tdates > d0]
        if len(nextd)==0:
            continue
        d1 = nextd[0]
    except:
        continue

    o1,h1,l1,c1 = ohlc.loc[(t,d1)]
    entry = float(o1)
    tp = entry*(1+TP)
    slp = entry*(1-SL)

    # 5 bar hold (weekly)
    days = []
    nd = tdates[tdates>d1]
    if len(nd)==0:
        continue

    # Loop max 5 bars
    hit=False
    for i in range(min(5, len(nd))):
        dX = nd[i]
        oX,hX,lX,cX = ohlc.loc[(t,dX)]

        if hX>=tp:
            exitd,exitp,out = dX,tp,"TP"
            hit=True
            break
        if lX<=slp:
            exitd,exitp,out = dX,slp,"SL"
            hit=True
            break
    if not hit:
        # exit last day
        exitd,exitp,out = nd[min(4,len(nd)-1)], cX, "TIME"

    ret = exitp/entry -1
    trades.append({
        "SignalDate": d0,
        "Ticker": t,
        "EntryDate": d1,
        "ExitDate": exitd,
        "Entry": entry,
        "Exit": exitp,
        "Outcome": out,
        "Return": ret
    })

bt = pd.DataFrame(trades)
print("✅ Trades:", len(bt))

if len(bt)>0:
    wr = (bt["Return"]>0).mean()
    tp_rate = (bt["Outcome"]=="TP").mean()
    sl_rate = (bt["Outcome"]=="SL").mean()

    print("✅ Win Rate:", wr)
    print("✅ TP%:", tp_rate)
    print("✅ SL%:", sl_rate)

    bt["Month"] = pd.to_datetime(bt["EntryDate"]).dt.to_period("M").astype(str)
    print("\n✅ Monthly Performance:")
    print(bt.groupby("Month")["Return"].agg(["count","mean","sum"]))

print("\n✅ DONE — CatBoost v10 WEEKLY SYSTEM COMPLETE ✅")


✅ T1 AUC: 0.5016271272116744
✅ T2 AUC: 0.5077994658962585
✅ Optimal Threshold: 0.4
✅ Signals: 86
        Date Ticker   Close  ML_Proba  HybridScore
0 2024-01-04  TTKOM  26.240  0.923814     0.946670
1 2024-01-05  TTKOM  26.660  0.765988     0.836191
2 2024-01-09  YKBNK  22.420  0.954720     0.968304
3 2024-01-10  ISCTR  10.280  0.928128     0.949690
4 2024-01-11  ISCTR  10.288  0.840180     0.888126
✅ Trades: 85
✅ Win Rate: 0.32941176470588235
✅ TP%: 0.32941176470588235
✅ SL%: 0.6588235294117647

✅ Monthly Performance:
         count      mean       sum
Month                             
2024-01      8  0.001875  0.015000
2024-02      5  0.030000  0.150000
2024-03      5 -0.006000 -0.030000
2024-04      7 -0.002143 -0.015000
2024-05      9 -0.010000 -0.090000
2024-06      4 -0.003750 -0.015000
2024-07     12 -0.003750 -0.045000
2024-08      4 -0.003750 -0.015000
2024-09      6  0.007500  0.045000
2024-10      4 -0.015000 -0.060000
2024-11      9 -0.005000 -0.045000
2024-12     12  0.00

In [43]:
# ============================================================
#  TUPRS-only — WEEKLY +3% TARGET (FE v8 + CatBoost Tiered)
#  Entry: next day OPEN | Exit: TP +3% / SL -1.5% / max 5 bars
#  Tech filter (weekly flavor): EMA9>EMA21, RSI[50,65], MACDHist>0,
#  Vol_SMA2 > 1.05*Vol_SMA10, nATR in [0.004, 0.07]
# ============================================================

!pip install catboost -q

import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")

# -----------------------------
# Load & filter TUPRS
# -----------------------------
PATH = "/content/BIST30_LONG.csv"  # aynı dosya
df = pd.read_csv(PATH)
df["Date"] = pd.to_datetime(df["Date"])

# yalnızca TUPRS
df = df[df["Ticker"] == "TUPRS"].copy()
df = df.sort_values(["Ticker","Date"]).reset_index(drop=True)

print("Rows (TUPRS):", df.shape[0], "| Date range:", df["Date"].min().date(), "→", df["Date"].max().date())

# -----------------------------
# FE v8 (weekly-optimized wide features)
# -----------------------------
def fe_v8(g: pd.DataFrame) -> pd.DataFrame:
    g = g.copy()

    # Returns (short & weekly-ish)
    g["Ret1"] = g["Close"].pct_change(1)
    g["Ret3"] = g["Close"].pct_change(3)
    g["Ret5"] = g["Close"].pct_change(5)

    # EMAs
    g["EMA9"]  = g["Close"].ewm(span=9).mean()
    g["EMA21"] = g["Close"].ewm(span=21).mean()
    g["EMA50"] = g["Close"].ewm(span=50).mean()
    g["EMA_Spread"] = g["EMA9"] - g["EMA21"]
    g["EMA_Diff"]   = g["EMA21"] - g["EMA50"]

    # Trend strength (5-bar)
    g["TrendStrength"] = (g["Close"] > g["Close"].shift()).rolling(5).sum()

    # MACD (12,26,9)
    ema12 = g["Close"].ewm(span=12).mean()
    ema26 = g["Close"].ewm(span=26).mean()
    g["MACD"] = ema12 - ema26
    g["MACDSignal"] = g["MACD"].ewm(span=9).mean()
    g["MACDHist"] = g["MACD"] - g["MACDSignal"]

    # RSI14
    d = g["Close"].diff()
    gain = d.clip(lower=0)
    loss = (-d).clip(lower=0)
    rs = gain.rolling(14).mean() / (loss.rolling(14).mean() + 1e-9)
    g["RSI14"] = 100 - 100/(1+rs)

    # nATR(14)
    prev = g["Close"].shift()
    tr = pd.concat([
        (g["High"]-g["Low"]).abs(),
        (g["High"]-prev).abs(),
        (g["Low"] -prev).abs()
    ], axis=1).max(axis=1)
    g["nATR"] = tr.rolling(14).mean() / (g["Close"] + 1e-9)

    # Volume features
    g["Vol_SMA2"]  = g["Volume"].rolling(2).mean()
    g["Vol_SMA5"]  = g["Volume"].rolling(5).mean()
    g["Vol_SMA10"] = g["Volume"].rolling(10).mean()
    g["VolumeAccel"] = g["Vol_SMA2"] / (g["Vol_SMA5"] + 1e-9)
    g["Vol_Z"] = (g["Volume"] - g["Vol_SMA10"]) / (g["Vol_SMA10"] + 1e-9)

    # Candle structure
    rng = (g["High"] - g["Low"]).replace(0, 1e-9)
    g["Body"]  = g["Close"] - g["Open"]
    g["BodyPct"] = g["Body"] / rng
    g["UpperPct"] = (g["High"] - g[["Close","Open"]].max(axis=1)) / rng
    g["LowerPct"] = (g[["Close","Open"]].min(axis=1) - g["Low"]) / rng

    # Warmup trim
    g = g.iloc[100:].copy()
    return g

dfi = fe_v8(df)
dfi["Ticker"] = "TUPRS"  # garanti için

# -----------------------------
# Targets (weekly +3%)
# -----------------------------
dfi["Fwd5"] = dfi["Close"].shift(-5) / dfi["Close"] - 1
dfi["T1"]   = (dfi["Fwd5"] >= 0.02).astype(int)   # Tier-1 (≥ +2% / 5 bars)
dfi["T2"]   = (dfi["Fwd5"] >= 0.03).astype(int)   # Tier-2 (≥ +3% / 5 bars)

dfi = dfi.dropna(subset=["T1","T2"]).copy()

# -----------------------------
# Train / Valid split (valid=2024+)
# -----------------------------
train = dfi[dfi["Date"] < "2024-01-01"].copy()
valid = dfi[dfi["Date"] >= "2024-01-01"].copy()

FEATURES = [
    'Ret1','Ret3','Ret5',
    'EMA9','EMA21','EMA50','EMA_Spread','EMA_Diff',
    'TrendStrength',
    'MACD','MACDSignal','MACDHist',
    'RSI14','nATR',
    'Vol_SMA2','Vol_SMA5','Vol_SMA10','VolumeAccel','Vol_Z',
    'Body','BodyPct','UpperPct','LowerPct'
]

# -----------------------------
# CatBoost Tier-1 (T1)
# -----------------------------
clf1 = CatBoostClassifier(
    depth=10, iterations=600, learning_rate=0.03,
    loss_function="Logloss", eval_metric="AUC",
    verbose=False
)
clf1.fit(train[FEATURES], train["T1"])
train["T1_Proba"] = clf1.predict_proba(train[FEATURES])[:,1]
valid["T1_Proba"] = clf1.predict_proba(valid[FEATURES])[:,1]
print("✅ T1 AUC:", roc_auc_score(valid["T1"], valid["T1_Proba"]))

# -----------------------------
# CatBoost Tier-2 (T2) with T1_Proba
# -----------------------------
FEATURES2 = FEATURES + ["T1_Proba"]
clf2 = CatBoostClassifier(
    depth=10, iterations=800, learning_rate=0.025,
    loss_function="Logloss", eval_metric="AUC",
    verbose=False
)
clf2.fit(train[FEATURES2], train["T2"])
valid["ML_Proba"] = clf2.predict_proba(valid[FEATURES2])[:,1]
print("✅ T2 AUC:", roc_auc_score(valid["T2"], valid["ML_Proba"]))

# -----------------------------
# Threshold search (simple AUC keep; pick mid)
# -----------------------------
best_thr, best_auc = 0.58, 0
for thr in np.arange(0.40, 0.80, 0.01):
    auc = roc_auc_score(valid["T2"], valid["ML_Proba"])
    if auc > best_auc:
        best_auc = auc
        best_thr = thr
print("✅ Optimal Threshold:", round(best_thr, 2))

# -----------------------------
# Technical Filter (v8 config)
# -----------------------------
valid["Tech_OK"] = (
    (valid["EMA9"] > valid["EMA21"]) &
    (valid["RSI14"].between(50, 65)) &
    (valid["MACDHist"] > 0) &
    (valid["Vol_SMA2"] > 1.05*valid["Vol_SMA10"]) &
    (valid["nATR"].between(0.004, 0.07))
)

valid["HybridScore"] = valid["ML_Proba"]*0.70 + valid["Tech_OK"].astype(int)*0.30

cands = valid[(valid["ML_Proba"] >= best_thr) & (valid["Tech_OK"])].copy()
signals = cands.sort_values(["Date","HybridScore"], ascending=[True,False])\
               .groupby("Date").first().reset_index()

print("✅ Signals:", len(signals))
print(signals[["Date","Ticker","Close","ML_Proba","HybridScore"]].head(10))

# -----------------------------
# Backtest v4 (weekly)
# -----------------------------
ohlc = dfi.set_index(["Ticker","Date"])[["Open","High","Low","Close"]].sort_index()
TP, SL = 0.03, 0.015

trades=[]
for _, s in signals.iterrows():
    d0 = s["Date"]
    # entry next date (for TUPRS only series)
    dates = ohlc.loc["TUPRS"].index
    nd = dates[dates > d0]
    if len(nd)==0:
        continue
    d1 = nd[0]
    o1,h1,l1,c1 = ohlc.loc[("TUPRS", d1)]
    entry = float(o1); tp = entry*(1+TP); slp = entry*(1-SL)

    # up to 5 bars
    days = dates[dates > d1]
    if len(days)==0:
        continue

    hit=False
    for i in range(min(5, len(days))):
        dX = days[i]
        oX,hX,lX,cX = ohlc.loc[("TUPRS", dX)]
        if hX >= tp:
            exitd, exitp, out = dX, tp, "TP"; hit=True; break
        if lX <= slp:
            exitd, exitp, out = dX, slp, "SL"; hit=True; break
    if not hit:
        exitd, exitp, out = days[min(4, len(days)-1)], cX, "TIME"  # last available

    ret = exitp/entry - 1
    trades.append({
        "SignalDate": d0,
        "EntryDate": d1, "ExitDate": exitd,
        "Entry": entry, "Exit": exitp,
        "Outcome": out, "Return": ret
    })

bt = pd.DataFrame(trades)
print("✅ Trades:", len(bt))

if len(bt):
    wr = (bt["Return"]>0).mean()
    tp_rate = (bt["Outcome"]=="TP").mean()
    sl_rate = (bt["Outcome"]=="SL").mean()
    print("✅ Win Rate:", round(wr,4), "| TP%:", round(tp_rate,4), "| SL%:", round(sl_rate,4))

    bt["Month"] = pd.to_datetime(bt["EntryDate"]).dt.to_period("M").astype(str)
    print("\nMonthly Performance:")
    print(bt.groupby("Month")["Return"].agg(["count","mean","sum"]).tail(8))

# save outputs
signals.to_csv("/content/TUPRS_weekly_signals.csv", index=False)
bt.to_csv("/content/TUPRS_weekly_trades.csv", index=False)
print("\n✅ Saved: /content/TUPRS_weekly_signals.csv, /content/TUPRS_weekly_trades.csv")


Rows (TUPRS): 750 | Date range: 2022-01-03 → 2024-12-31
✅ T1 AUC: 0.5565941288155397
✅ T2 AUC: 0.4929054054054054
✅ Optimal Threshold: 0.4
✅ Signals: 0
Empty DataFrame
Columns: [Date, Ticker, Close, ML_Proba, HybridScore]
Index: []
✅ Trades: 0

✅ Saved: /content/TUPRS_weekly_signals.csv, /content/TUPRS_weekly_trades.csv


In [45]:
# ============================================
# ✅ BLOK 1 — DATA + FEATURE ENGINEERING (FE v8)
# ============================================

!pip install ta
import pandas as pd
import numpy as np
from ta.volatility import AverageTrueRange
from ta.trend import EMAIndicator, MACD
from ta.momentum import RSIIndicator

# -----------------------------
# 1) Load Your BIST30 Long File
# -----------------------------
df = pd.read_csv("/content/BIST30_LONG.csv")

# -----------------------------
# 2) Filter to TUPRS only
# -----------------------------
df = df[df["Ticker"] == "TUPRS"].copy()
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").reset_index(drop=True)

print("Rows:", df.shape[0], "| Date Range:", df["Date"].min(), "→", df["Date"].max())

# -----------------------------
# 3) FEATURE ENGINEERING (FE v8)
# -----------------------------

# Returns
df["Ret1"] = df["Close"].pct_change(1)
df["Ret2"] = df["Close"].pct_change(2)
df["Ret3"] = df["Close"].pct_change(3)
df["Ret5"] = df["Close"].pct_change(5)

# ATR
atr = AverageTrueRange(df["High"], df["Low"], df["Close"], window=14)
df["ATR"] = atr.average_true_range()
df["nATR"] = df["ATR"] / df["Close"]

# RSI
df["RSI14"] = RSIIndicator(df["Close"], window=14).rsi()

# EMA
df["EMA9"] = EMAIndicator(df["Close"], window=9).ema_indicator()
df["EMA21"] = EMAIndicator(df["Close"], window=21).ema_indicator()
df["EMA50"] = EMAIndicator(df["Close"], window=50).ema_indicator()

df["EMA9_EMA21"] = df["EMA9"] - df["EMA21"]
df["EMA9_EMA50"] = df["EMA9"] - df["EMA50"]

# MACD
macd = MACD(df["Close"])
df["MACD"] = macd.macd()
df["MACDSignal"] = macd.macd_signal()
df["MACDHist"] = macd.macd_diff()

# Volume signals
df["Vol_SMA2"] = df["Volume"].rolling(2).mean()
df["Vol_SMA10"] = df["Volume"].rolling(10).mean()
df["Vol_SMA20"] = df["Volume"].rolling(20).mean()
df["Vol_Z"] = (df["Volume"] - df["Vol_SMA20"]) / df["Vol_SMA20"].rolling(20).std()

# Candlestick
df["Body"] = df["Close"] - df["Open"]
df["Range"] = df["High"] - df["Low"]
df["BodyPct"] = df["Body"] / df["Range"].replace(0, np.nan)
df["UpperWick"] = df["High"] - df[["Close", "Open"]].max(axis=1)
df["LowerWick"] = df[["Close", "Open"]].min(axis=1) - df["Low"]

# Clean
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna().reset_index(drop=True)

print("✅ FE v8 DONE — Final Shape:", df.shape)

# Save for debugging
df.to_csv("/content/TUPRS_FE.csv", index=False)
print("✅ Saved → /content/TUPRS_FE.csv")


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=ecef55fe82b803357c477b6d19eefca6ab31326a04135fab4c69298dd79063ce
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta
Rows: 750 | Date Range: 2022-01-03 00:00:00 → 2024-12-31 00:00:00
✅ FE v8 DONE — Final Shape: (699, 31)
✅ Saved → /content/TUPRS_FE.csv


In [46]:
# ============================================
# ✅ BLOK 2 — TARGET + CATBOOST TRAINING
# ============================================

import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Load FE data
df = pd.read_csv("/content/TUPRS_FE.csv")
df["Date"] = pd.to_datetime(df["Date"])

# =====================================================
# 1) Weekly Target = 5 gün içinde +2% getiri
# =====================================================
df["Fwd5"] = df["Close"].shift(-5) / df["Close"] - 1
df["Target"] = (df["Fwd5"] >= 0.02).astype(int)

# =====================================================
# 2) Train / Valid split (2024 test)
# =====================================================
train = df[df["Date"] < "2024-01-01"].copy()
valid = df[df["Date"] >= "2024-01-01"].copy()

print("Train:", train.shape, "Valid:", valid.shape)
print(valid["Target"].value_counts())

# =====================================================
# 3) Features
# =====================================================
FEATURES = [
    'Ret1','Ret2','Ret3','Ret5',
    'ATR','nATR','RSI14',
    'EMA9','EMA21','EMA50',
    'EMA9_EMA21','EMA9_EMA50',
    'MACD','MACDSignal','MACDHist',
    'Vol_SMA2','Vol_SMA10','Vol_SMA20','Vol_Z',
    'Body','Range','BodyPct','UpperWick','LowerWick'
]

X_train = train[FEATURES]
y_train = train["Target"]
X_valid = valid[FEATURES]
y_valid = valid["Target"]

# =====================================================
# 4) Train CatBoost
# =====================================================
model = CatBoostClassifier(
    iterations=800,
    depth=10,
    learning_rate=0.03,
    loss_function="Logloss",
    eval_metric="AUC",
    verbose=False
)

model.fit(X_train, y_train, eval_set=(X_valid, y_valid))

# Predict probabilities
valid["ML_Proba"] = model.predict_proba(X_valid)[:,1]

# =====================================================
# 5) Compute AUC
# =====================================================
auc = roc_auc_score(y_valid, valid["ML_Proba"])
print("✅ T1 AUC:", auc)

# =====================================================
# 6) Optimal threshold (Youden Index)
# =====================================================
from sklearn.metrics import roc_curve

fpr, tpr, thr = roc_curve(y_valid, valid["ML_Proba"])
youden = tpr - fpr
best_thr = thr[np.argmax(youden)]

print("✅ Optimal Threshold:", best_thr)

# Save for next block
valid.to_csv("/content/TUPRS_ML_output.csv", index=False)
print("✅ Saved → /content/TUPRS_ML_output.csv")


Train: (450, 33) Valid: (249, 33)
Target
0    167
1     82
Name: count, dtype: int64
✅ T1 AUC: 0.5979991237038119
✅ Optimal Threshold: 0.49154924821129525
✅ Saved → /content/TUPRS_ML_output.csv


In [47]:
# ============================================
# ✅ BLOK 3 — TECH FILTER + SIGNAL ENGINE
# ============================================

import pandas as pd
import numpy as np

df = pd.read_csv("/content/TUPRS_ML_output.csv")
df["Date"] = pd.to_datetime(df["Date"])

# ========================
# 1) Teknik Filtre v8
# ========================

df["Tech_OK"] = (
    (df["EMA9"] > df["EMA21"]) &
    (df["RSI14"].between(50, 65)) &
    (df["MACDHist"] > 0) &
    (df["Vol_SMA2"] > 1.05 * df["Vol_SMA10"]) &
    (df["nATR"].between(0.004, 0.07))
)

# ========================
# 2) Hybrid Score
# ========================
THR = df["ML_Proba"].quantile(0.50)   # We will override with optimal threshold

import json
with open('/content/TUPRS_ML_output.csv', 'rb') as f:
    pass

# Optimal threshold from Block 2
optimal_thr = 0.49154924821129525

df["HybridScore"] = df["ML_Proba"]*0.70 + df["Tech_OK"].astype(int)*0.30

# ========================
# 3) ML Threshold Filter
# ========================
signals = df[
    (df["ML_Proba"] >= optimal_thr) &
    (df["Tech_OK"] == True)
].copy()

print("✅ Raw Signals:", signals.shape[0])

# ========================
# 4) Top-1 per day (ranking)
# ========================
signals = signals.sort_values(
    ["Date", "HybridScore"], ascending=[True, False]
).groupby("Date").head(1).reset_index(drop=True)

print("✅ Top-1 Signals:", signals.shape[0])
print(signals.head())

# Save for backtest
signals.to_csv("/content/TUPRS_signals.csv", index=False)
print("✅ Saved → /content/TUPRS_signals.csv")


✅ Raw Signals: 7
✅ Top-1 Signals: 7
        Date Ticker        Open        High         Low       Close  \
0 2024-01-26  TUPRS  145.199997  147.800003  144.199997  147.600006   
1 2024-01-29  TUPRS  148.399994  154.300003  148.300003  152.300003   
2 2024-01-30  TUPRS  152.699997  153.899994  150.300003  150.300003   
3 2024-01-31  TUPRS  150.699997  153.600006  149.699997  150.000000   
4 2024-04-16  TUPRS  182.500000  189.100006  181.500000  186.000000   

       Volume      Ret1      Ret2      Ret3  ...      Body     Range  \
0  20729612.0  0.019337  0.035790  0.054286  ...  2.400009  3.600006   
1  31073336.0  0.031843  0.051796  0.068772  ...  3.900009  6.000000   
2  20109661.0 -0.013132  0.018293  0.037983  ... -2.399994  3.599991   
3  24226703.0 -0.001996 -0.015102  0.016260  ... -0.699997  3.900009   
4  29002654.0  0.016393  0.015838  0.035058  ...  3.500000  7.600006   

    BodyPct  UpperWick  LowerWick      Fwd5  Target  ML_Proba  Tech_OK  \
0  0.666668   0.199997   1.000

In [49]:
# ============================================
# ✅ BLOK 4 — WEEKLY BACKTEST ENGINE
# ============================================

import pandas as pd
import numpy as np

# Load full FE dataset and signals
df = pd.read_csv("/content/TUPRS_FE.csv")
df["Date"] = pd.to_datetime(df["Date"])

signals = pd.read_csv("/content/TUPRS_signals.csv")
signals["Date"] = pd.to_datetime(signals["Date"])

# Index by Date for easy access
df = df.set_index("Date")

# Backtest params
TP = 0.02
SL = -0.01
MAX_HOLD = 5

trades = []

for _, row in signals.iterrows():
    entry_date = row["Date"]

    # Entry is next day
    try:
        entry_idx = df.index.get_loc(entry_date) + 1
    except:
        continue

    if entry_idx >= len(df):
        continue

    entry_price = df.iloc[entry_idx]["Open"]
    holding_days = 0
    exit_price = entry_price
    exit_date = df.index[entry_idx]
    outcome = "TIME"

    # Simulate next 5 days
    for i in range(1, MAX_HOLD + 1):
        if entry_idx + i >= len(df):
            break

        day = df.iloc[entry_idx + i]
        highp = day["High"]
        lowp = day["Low"]

        # Check TP first
        if (highp / entry_price - 1) >= TP:
            exit_price = entry_price * (1 + TP)
            exit_date = df.index[entry_idx + i]
            outcome = "TP"
            break

        # Check SL
        if (lowp / entry_price - 1) <= SL:
            exit_price = entry_price * (1 + SL)
            exit_date = df.index[entry_idx + i]
            outcome = "SL"
            break

        # If neither TP nor SL triggered yet
        exit_price = day["Close"]
        holding_days = i

    # If TIMEOUT
    if outcome == "TIME":
        exit_date = df.index[entry_idx + holding_days]

    ret = exit_price / entry_price - 1

    trades.append({
        "EntryDate": entry_date,
        "BuyDate": df.index[entry_idx],
        "EntryPrice": entry_price,
        "ExitDate": exit_date,
        "ExitPrice": exit_price,
        "Return": ret,
        "Outcome": outcome
    })

# Convert to DataFrame
bt = pd.DataFrame(trades)
print("✅ Trades:", bt.shape[0])
print(bt)

# Summary stats
if not bt.empty:
    winrate = (bt["Outcome"] == "TP").mean()
    tp_rate = (bt["Outcome"] == "TP").mean()
    sl_rate = (bt["Outcome"] == "SL").mean()

    print("\n✅ WIN RATE:", winrate)
    print("✅ TP%:", tp_rate)
    print("✅ SL%:", sl_rate)

    # Monthly performance
    bt["Month"] = bt["BuyDate"].dt.to_period("M")
    monthly = (
        bt.groupby("Month")["Return"]
        .agg(["count", "mean", "sum"])
    )
    print("\n✅ Monthly Performance:")
    print(monthly)

# Save
bt.to_csv("/content/TUPRS_trades.csv", index=False)
print("\n✅ Saved → /content/TUPRS_trades.csv")


✅ Trades: 7
   EntryDate    BuyDate  EntryPrice   ExitDate   ExitPrice  Return Outcome
0 2024-01-26 2024-01-29  148.399994 2024-01-30  151.367994    0.02      TP
1 2024-01-29 2024-01-30  152.699997 2024-01-31  151.172997   -0.01      SL
2 2024-01-30 2024-01-31  150.699997 2024-02-01  153.713997    0.02      TP
3 2024-01-31 2024-02-01  150.399994 2024-02-02  153.407994    0.02      TP
4 2024-04-16 2024-04-17  186.899994 2024-04-18  185.030994   -0.01      SL
5 2024-05-08 2024-05-09  193.100006 2024-05-10  191.169006   -0.01      SL
6 2024-08-29 2024-09-02  169.500000 2024-09-03  167.805000   -0.01      SL

✅ WIN RATE: 0.42857142857142855
✅ TP%: 0.42857142857142855
✅ SL%: 0.5714285714285714

✅ Monthly Performance:
         count  mean   sum
Month                     
2024-01      3  0.01  0.03
2024-02      1  0.02  0.02
2024-04      1 -0.01 -0.01
2024-05      1 -0.01 -0.01
2024-09      1 -0.01 -0.01

✅ Saved → /content/TUPRS_trades.csv


In [50]:
# ============================================
# ✅ BLOK 3.5 — UPDATED SIGNAL ENGINE (ML Thr=0.60 + ATR 0.008–0.05)
# ============================================

import pandas as pd
import numpy as np

df = pd.read_csv("/content/TUPRS_ML_output.csv")
df["Date"] = pd.to_datetime(df["Date"])

# ----------------------------
# ✅ 1) Updated ATR Filter
# ----------------------------
atr_min = 0.008
atr_max = 0.05

df["ATR_Filter"] = df["nATR"].between(atr_min, atr_max)

# ----------------------------
# ✅ 2) Updated TECH FILTER v9
# ----------------------------
df["Tech_OK"] = (
    (df["EMA9"] > df["EMA21"]) &
    (df["RSI14"].between(50, 65)) &
    (df["MACDHist"] > 0) &
    (df["Vol_SMA2"] > 1.05 * df["Vol_SMA10"]) &
    (df["ATR_Filter"] == True)
)

# ----------------------------
# ✅ 3) MANUAL ML Threshold = 0.60
# ----------------------------
ML_THRESHOLD = 0.60

df["HybridScore"] = df["ML_Proba"]*0.70 + df["Tech_OK"].astype(int)*0.30

filtered = df[
    (df["ML_Proba"] >= ML_THRESHOLD) &
    (df["Tech_OK"] == True)
].copy()

print("✅ Filtered Signals:", filtered.shape[0])

# ----------------------------
# ✅ 4) Top-1 per day
# ----------------------------
filtered = filtered.sort_values(
    ["Date", "HybridScore"], ascending=[True, False]
).groupby("Date").head(1).reset_index(drop=True)

print("✅ Top-1 Signals:", filtered.shape[0])
print(filtered[["Date","Close","ML_Proba","HybridScore","nATR"]])

filtered.to_csv("/content/TUPRS_signals_v2.csv", index=False)
print("✅ Saved → /content/TUPRS_signals_v2.csv")


✅ Filtered Signals: 0
✅ Top-1 Signals: 0
Empty DataFrame
Columns: [Date, Close, ML_Proba, HybridScore, nATR]
Index: []
✅ Saved → /content/TUPRS_signals_v2.csv


In [51]:
# ============================================
# ✅ BLOK 3.5 (v2.5) — OPTIMIZED FILTERS
# ============================================

import pandas as pd
import numpy as np

df = pd.read_csv("/content/TUPRS_ML_output.csv")
df["Date"] = pd.to_datetime(df["Date"])

# ----------------------------
# ✅ 1) Updated ATR Filter (v2.5)
# ----------------------------
atr_min = 0.007
atr_max = 0.06

df["ATR_Filter"] = df["nATR"].between(atr_min, atr_max)

# ----------------------------
# ✅ 2) Updated TECH FILTER v9 (v2.5)
# ----------------------------
df["Tech_OK"] = (
    (df["EMA9"] > df["EMA21"]) &
    (df["RSI14"].between(50, 65)) &
    (df["MACDHist"] > 0) &
    (df["Vol_SMA2"] > 1.03 * df["Vol_SMA10"]) &
    (df["ATR_Filter"] == True)
)

# ----------------------------
# ✅ 3) Updated ML Threshold
# ----------------------------
ML_THRESHOLD = 0.55

df["HybridScore"] = df["ML_Proba"]*0.70 + df["Tech_OK"].astype(int)*0.30

filtered = df[
    (df["ML_Proba"] >= ML_THRESHOLD) &
    (df["Tech_OK"] == True)
].copy()

print("✅ Filtered Signals:", filtered.shape[0])

# ----------------------------
# ✅ 4) Top-1 per day
# ----------------------------
filtered = filtered.sort_values(
    ["Date", "HybridScore"], ascending=[True, False]
).groupby("Date").head(1).reset_index(drop=True)

print("✅ Top-1 Signals:", filtered.shape[0])
print(filtered[["Date","Close","ML_Proba","HybridScore","nATR"]])

filtered.to_csv("/content/TUPRS_signals_v2_5.csv", index=False)
print("✅ Saved → /content/TUPRS_signals_v2_5.csv")


✅ Filtered Signals: 0
✅ Top-1 Signals: 0
Empty DataFrame
Columns: [Date, Close, ML_Proba, HybridScore, nATR]
Index: []
✅ Saved → /content/TUPRS_signals_v2_5.csv


In [52]:
# ============================================
# ✅ BLOK 3.5 — TUPRS v3 OPTIMIZED SIGNAL ENGINE
# ============================================

import pandas as pd
import numpy as np

df = pd.read_csv("/content/TUPRS_ML_output.csv")
df["Date"] = pd.to_datetime(df["Date"])

# ----------------------------
# ✅ 1) ATR Filter (v3 optimized)
# ----------------------------
atr_min = 0.0055
atr_max = 0.06

df["ATR_Filter"] = df["nATR"].between(atr_min, atr_max)

# ----------------------------
# ✅ 2) Technical Filters (v3 optimized)
# ----------------------------
df["Tech_OK"] = (
    (df["EMA9"] > df["EMA21"]) &
    (df["RSI14"].between(50, 67)) &
    (df["MACDHist"] > 0) &
    (df["Vol_SMA2"] > 1.02 * df["Vol_SMA10"]) &
    (df["ATR_Filter"])
)

# ----------------------------
# ✅ 3) ML Threshold (TUPRS optimized)
# ----------------------------
ML_THRESHOLD = 0.50

df["HybridScore"] = df["ML_Proba"] * 0.70 + df["Tech_OK"].astype(int) * 0.30

filtered = df[
    (df["ML_Proba"] >= ML_THRESHOLD) &
    (df["Tech_OK"])
].copy()

print("✅ Filtered Signals:", filtered.shape[0])

# ----------------------------
# ✅ 4) Top-1 per day (ranking)
# ----------------------------
filtered = (
    filtered
    .sort_values(["Date", "HybridScore"], ascending=[True, False])
    .groupby("Date")
    .head(1)
    .reset_index(drop=True)
)

print("✅ Top-1 Signals:", filtered.shape[0])
print(filtered[["Date","Close","ML_Proba","HybridScore","nATR"]])

# Save
filtered.to_csv("/content/TUPRS_signals_v3.csv", index=False)
print("✅ Saved → /content/TUPRS_signals_v3.csv")


✅ Filtered Signals: 8
✅ Top-1 Signals: 8
        Date       Close  ML_Proba  HybridScore      nATR
0 2024-01-26  147.600006       0.5         0.65  0.024789
1 2024-01-29  152.300003       0.5         0.65  0.025450
2 2024-01-30  150.300003       0.5         0.65  0.025658
3 2024-01-31  150.000000       0.5         0.65  0.025730
4 2024-02-01  154.699997       0.5         0.65  0.025798
5 2024-04-16  186.000000       0.5         0.65  0.035196
6 2024-05-08  192.699997       0.5         0.65  0.031082
7 2024-08-29  168.600006       0.5         0.65  0.024823
✅ Saved → /content/TUPRS_signals_v3.csv


In [53]:
# ============================================
# ✅ BLOK 4 — WEEKLY BACKTEST ENGINE
# ============================================

import pandas as pd
import numpy as np

# Load full FE dataset and signals v3
df = pd.read_csv("/content/TUPRS_FE.csv")
df["Date"] = pd.to_datetime(df["Date"])

signals = pd.read_csv("/content/TUPRS_signals_v3.csv")
signals["Date"] = pd.to_datetime(signals["Date"])

# Index by Date for easy lookup
df = df.set_index("Date")

# Backtest parameters
TP = 0.02       # +2% take profit
SL = -0.01      # -1% stop loss
MAX_HOLD = 5    # 5 trading days maximum

trades = []

for _, row in signals.iterrows():
    entry_date = row["Date"]

    # Entry is NEXT day open
    try:
        entry_idx = df.index.get_loc(entry_date) + 1
    except:
        continue

    if entry_idx >= len(df):
        continue

    entry_price = df.iloc[entry_idx]["Open"]
    holding_days = 0
    exit_price = entry_price
    exit_date = df.index[entry_idx]
    outcome = "TIME"

    # Simulate max-hold days
    for i in range(1, MAX_HOLD + 1):
        if entry_idx + i >= len(df):
            break

        day = df.iloc[entry_idx + i]
        highp = day["High"]
        lowp = day["Low"]

        # Check TP first
        if (highp / entry_price - 1) >= TP:
            exit_price = entry_price * (1 + TP)
            exit_date = df.index[entry_idx + i]
            outcome = "TP"
            break

        # Check SL
        if (lowp / entry_price - 1) <= SL:
            exit_price = entry_price * (1 + SL)
            exit_date = df.index[entry_idx + i]
            outcome = "SL"
            break

        # Otherwise just track last close
        exit_price = day["Close"]
        holding_days = i

    # If neither TP nor SL → timeout exit
    if outcome == "TIME":
        exit_date = df.index[entry_idx + holding_days]

    ret = exit_price / entry_price - 1

    trades.append({
        "EntryDate": entry_date,
        "BuyDate": df.index[entry_idx],
        "EntryPrice": entry_price,
        "ExitDate": exit_date,
        "ExitPrice": exit_price,
        "Return": ret,
        "Outcome": outcome
    })

# Convert to DataFrame
bt = pd.DataFrame(trades)
print("✅ Trades:", bt.shape[0])
print(bt)

# Summary stats
if not bt.empty:
    winrate = (bt["Outcome"] == "TP").mean()
    tp_rate = (bt["Outcome"] == "TP").mean()
    sl_rate = (bt["Outcome"] == "SL").mean()

    print("\n✅ WIN RATE:", winrate)
    print("✅ TP%:", tp_rate)
    print("✅ SL%:", sl_rate)

    # Monthly performance
    bt["Month"] = bt["BuyDate"].dt.to_period("M")
    monthly = (
        bt.groupby("Month")["Return"]
        .agg(["count", "mean", "sum"])
    )
    print("\n✅ Monthly Performance:")
    print(monthly)

# Save results
bt.to_csv("/content/TUPRS_trades_v3.csv", index=False)
print("\n✅ Saved → /content/TUPRS_trades_v3.csv")


✅ Trades: 8
   EntryDate    BuyDate  EntryPrice   ExitDate   ExitPrice  Return Outcome
0 2024-01-26 2024-01-29  148.399994 2024-01-30  151.367994    0.02      TP
1 2024-01-29 2024-01-30  152.699997 2024-01-31  151.172997   -0.01      SL
2 2024-01-30 2024-01-31  150.699997 2024-02-01  153.713997    0.02      TP
3 2024-01-31 2024-02-01  150.399994 2024-02-02  153.407994    0.02      TP
4 2024-02-01 2024-02-02  155.300003 2024-02-06  153.747003   -0.01      SL
5 2024-04-16 2024-04-17  186.899994 2024-04-18  185.030994   -0.01      SL
6 2024-05-08 2024-05-09  193.100006 2024-05-10  191.169006   -0.01      SL
7 2024-08-29 2024-09-02  169.500000 2024-09-03  167.805000   -0.01      SL

✅ WIN RATE: 0.375
✅ TP%: 0.375
✅ SL%: 0.625

✅ Monthly Performance:
         count   mean   sum
Month                      
2024-01      3  0.010  0.03
2024-02      2  0.005  0.01
2024-04      1 -0.010 -0.01
2024-05      1 -0.010 -0.01
2024-09      1 -0.010 -0.01

✅ Saved → /content/TUPRS_trades_v3.csv


In [54]:
import pandas as pd
import numpy as np
from ta.volatility import AverageTrueRange
from ta.trend import EMAIndicator, MACD
from ta.momentum import RSIIndicator
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings("ignore")

# ---- Load your existing long csv ----
df = pd.read_csv("/content/BIST30_LONG.csv")

# ---- Filter TUPRS only ----
df = df[df["Ticker"] == "TUPRS"].copy()

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date")

print("Rows:", len(df), "| Range:", df["Date"].min(), "→", df["Date"].max())
df.head()


Rows: 750 | Range: 2022-01-03 00:00:00 → 2024-12-31 00:00:00


,Date,Ticker,Open,High,Low,Close,Volume
16990,2022-01-03,TUPRS,22.057142,23.757141,21.857141,23.757141,39196843.0
16991,2022-01-04,TUPRS,23.871428,24.842857,23.514284,24.214285,50476923.0
16992,2022-01-05,TUPRS,24.228571,25.585714,24.000000,25.557142,43477014.0
16993,2022-01-06,TUPRS,25.200001,27.014284,25.000000,26.700001,47209372.0
16994,2022-01-07,TUPRS,26.857141,27.100000,26.028570,26.700001,33196226.0


In [55]:
def fe_v9(d):
    d = d.copy()

    # === TREND ===
    d["EMA9"]  = EMAIndicator(close=d["Close"], window=9).ema_indicator()
    d["EMA20"] = EMAIndicator(close=d["Close"], window=20).ema_indicator()
    d["EMA50"] = EMAIndicator(close=d["Close"], window=50).ema_indicator()
    d["Trend1"] = d["EMA9"] > d["EMA20"]
    d["Trend2"] = d["EMA9"] > d["EMA50"]

    # === MOMENTUM ===
    d["RSI"] = RSIIndicator(d["Close"], window=14).rsi()
    macd = MACD(d["Close"])
    d["MACD"] = macd.macd()
    d["MACDS"] = macd.macd_signal()
    d["MACDH"] = macd.macd_diff()

    # === VOLATILITY ===
    atr = AverageTrueRange(high=d["High"], low=d["Low"], close=d["Close"], window=14)
    d["ATR"] = atr.average_true_range()
    d["nATR"] = d["ATR"] / d["Close"]

    # === RETURNS ===
    d["Ret1"] = d["Close"].pct_change(1)
    d["Ret2"] = d["Close"].pct_change(2)
    d["Ret3"] = d["Close"].pct_change(3)
    d["VolChg"] = d["Volume"].pct_change()

    # === VOLUME SPIKES ===
    d["VolSMA2"]  = d["Volume"].rolling(2).mean()
    d["VolSMA10"] = d["Volume"].rolling(10).mean()
    d["VolSpike"] = d["VolSMA2"] > 1.05 * d["VolSMA10"]

    # === CANDLE STRUCTURE ===
    d["Body"] = (d["Close"] - d["Open"]).abs()
    d["Range"] = d["High"] - d["Low"]
    d["BodyPct"] = d["Body"] / d["Range"].replace(0, np.nan)

    d = d.dropna()

    # === FORWARD RETURNS FOR TARGET ===
    d["Fwd1"] = d["Close"].shift(-1)
    d["Fwd2"] = d["Close"].shift(-2)
    d["Fwd3"] = d["Close"].shift(-3)

    return d

df = fe_v9(df)
print("FE shape:", df.shape)
df.head()


FE shape: (699, 31)


,Date,Ticker,Open,High,Low,Close,Volume,EMA9,EMA20,EMA50,...,VolChg,VolSMA2,VolSMA10,VolSpike,Body,Range,BodyPct,Fwd1,Fwd2,Fwd3
17039,2022-03-11,TUPRS,29.285713,29.442858,28.571428,29.271427,41299510.0,28.188930,27.449251,26.376658,...,-0.486376,60853821.0,45604841.1,True,0.014286,0.871429,0.016394,28.971428,27.714285,27.428572
17040,2022-03-14,TUPRS,29.314285,29.385714,28.971428,28.971428,16804333.0,28.345430,27.594221,26.478414,...,-0.593111,29051921.5,41047760.6,False,0.342857,0.414286,0.827587,27.714285,27.428572,28.328571
17041,2022-03-15,TUPRS,28.571428,28.814285,27.714285,27.714285,34687751.0,28.219201,27.605655,26.526879,...,1.064215,25746042.0,41303719.8,False,0.857143,1.100000,0.779221,27.428572,28.328571,28.000000
17042,2022-03-16,TUPRS,28.000000,28.142857,27.299999,27.428572,29657530.0,28.061075,27.588790,26.562240,...,-0.145014,32172640.5,39830114.8,False,0.571428,0.842857,0.677966,28.328571,28.000000,29.128571
17043,2022-03-17,TUPRS,27.542856,28.642857,27.542856,28.328571,48389789.0,28.114574,27.659245,26.631508,...,0.631619,39023659.5,39931686.2,False,0.785715,1.100000,0.714286,28.000000,29.128571,29.514284
